# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a given youtube video.

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).


## Install OpenPose

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. -DUSE_CUDNN=OFF && make -j`nproc`
  


Selecting previously unselected package libgflags2.2.
(Reading database ... 128215 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.2-1build1_amd64.deb ...
Unpacking libgflags2.2 (2.2.2-1build1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.2-1build1_amd64.deb ...
Unpacking libgflags-dev (2.2.2-1build1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.4.0-1build1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.4.0-1build1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.4.0-1build1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.4.0-1build1) ...
Selecting previously unselected package libleveldb1d:amd64.
Preparing to unpack .../04-libleveldb1d_1.22-3ubuntu2_amd64.deb ...
Unpacking libleveldb1d:amd64 (1.22-3ubuntu2) ...
Selecting previously unselected package libleveldb-dev

## Detect poses on a test video

We are going to detect poses on the following youtube video:

Download the above youtube video, cut the first 5 seconds and do the pose detection on that 5 seconds:

Finally, visualize the result:

In [ ]:

from google.colab import drive

import os
drive.mount('/content/drive')

!cd openpose && ./build/examples/openpose/openpose.bin --video /content/drive/MyDrive/feed_test.mp4 --write_json ../output/ --part_candidates --display 0  --render_pose 0 --number_people_max 1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 38.603688 seconds.


In [ ]:
#Normalization

from google.colab import drive
import json
from math import sqrt
import os
pat="/content/output/"
os.makedirs('/content/out/', exist_ok=True)
path_out="/content/out/"
lof = os.listdir(pat)#['batata.json', 'sese.json']
for i in range(len(lof)):
  if lof[i] ==".ipynb_checkpoints":
    lof.remove(lof[i])
    break
print(lof)
#j=1
for j in range(len(lof)):
   
    f=open(pat+lof[j],'r')
    x= json.loads( f.read())

    pts=x['part_candidates'][0]
    orignx=pts["1"][0]
    origny=pts["1"][1]
    
    const =sqrt(((orignx-pts["8"][0])**2)+((origny-pts["8"][1])**2))
    for i in range(0,25):
        if ( pts[str(i)] !=[]):
            pts[str(i)][0]=(pts[ str(i) ][0]-orignx)/const
            pts[str(i)][1]=(pts[ str(i) ][1]-origny)/const
   # print(pts)
    
    f2=open(path_out+lof[j],'w')
    jprint=json.dumps(pts)
    f2.seek(0)
    f2.write('{"part_candidates":[')
    f2.write(jprint)
    f2.write("]}")
    f2.truncate()
    f2.close()
    print(j)

['feed_test_000000000074_keypoints.json', 'feed_test_000000000261_keypoints.json', 'feed_test_000000000032_keypoints.json', 'feed_test_000000000243_keypoints.json', 'feed_test_000000000299_keypoints.json', 'feed_test_000000000127_keypoints.json', 'feed_test_000000000183_keypoints.json', 'feed_test_000000000217_keypoints.json', 'feed_test_000000000143_keypoints.json', 'feed_test_000000000067_keypoints.json', 'feed_test_000000000173_keypoints.json', 'feed_test_000000000126_keypoints.json', 'feed_test_000000000103_keypoints.json', 'feed_test_000000000310_keypoints.json', 'feed_test_000000000196_keypoints.json', 'feed_test_000000000311_keypoints.json', 'feed_test_000000000253_keypoints.json', 'feed_test_000000000163_keypoints.json', 'feed_test_000000000113_keypoints.json', 'feed_test_000000000277_keypoints.json', 'feed_test_000000000134_keypoints.json', 'feed_test_000000000210_keypoints.json', 'feed_test_000000000197_keypoints.json', 'feed_test_000000000231_keypoints.json', 'feed_test_0000

In [ ]:
#finallllllllll one

# import necessary libraries
from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import numpy as np
import os
import json
from math import sqrt
from google.colab import drive
import cv2 as cv

# mount Google Drive
drive.mount('/content/drive')

# set paths for directories containing exercise data
pats=["/content/drive/MyDrive/jump_bary/","/content/drive/MyDrive/squat_bary/","/content/drive/MyDrive/jogging_bary/"]
pat="/content/out/"
out_path="/content/output/"
out=[]
lof = sorted(os.listdir(out_path))#['batata.json', 'sese.json']
for i in range(len(lof)):
  if lof[i] ==".ipynb_checkpoints":
    lof.remove(lof[i])
    break

for j in range(len(lof)):
    
    f=open(out_path+lof[j],'r+')
    pts= json.loads( f.read())

    out.append(pts['part_candidates'][0])


# get list of files in the first directory
x = sorted(os.listdir(pat))

# get lists of files in the other directories
y = []
y.append(sorted(os.listdir(pats[0])))
y.append(sorted(os.listdir(pats[1])))
y.append(sorted(os.listdir(pats[2])))

# remove any files that are not JSON files
for i in range(len(x)):
  if x[i] ==".ipynb_checkpoints":
    x.remove(x[i])
    break
for j in range(len(y)):
  for i in range(len(y[j])):
    if y[j][i] ==".ipynb_checkpoints":
      y[j].remove(y[j][i])
      break

# initialize empty lists
cls=[]
paths=[]
def error(wrong_joint_x, wrong_joint_y, frame):
    # This function draws a red circle at the position of the wrong joint in the given video frame
    radius = 10
    color = (0, 0, 255)  # red in BGR format
    thickness = -1  # fill circle
    cv.circle(frame, (int(wrong_joint_x),int( wrong_joint_y)), radius, color, thickness)
    return frame


def dp(dist_mat):
    """
    Find minimum-cost path through matrix dist_mat using dynamic programming.

    The cost of a path is defined as the sum of the matrix entries on that
    path. See the following for details of the algorithm:

    - http://en.wikipedia.org/wiki/Dynamic_time_warping
    - https://www.ee.columbia.edu/~dpwe/resources/matlab/dtw/dp.m

    The notation in the first reference was followed, while Dan Ellis's code
    (second reference) was used to check for correctness. Returns a list of
    path indices and the cost matrix.
    """
    # This function implements dynamic programming to find the minimum-cost path through a distance matrix.
    # It returns a list of path indices and the cost matrix.

    N, M = dist_mat.shape

    # Initialize the cost matrix
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1, N + 1):
        cost_mat[i, 0] = np.inf
    for i in range(1, M + 1):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_mat[i, j],  # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = N - 1
    j = M - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))

    # Strip infinity edges from cost_mat before returning
    cost_mat = cost_mat[1:, 1:]
    return (path[::-1], cost_mat)


def subf(a, b, pat2):
    # Open file a and read its contents into a JSON object
    f = open(pat + a, 'r')
    x3 = json.loads(f.read())
    # Extract the part candidates dictionary from the JSON object
    orign = x3['part_candidates'][0]
    f.close()

    # Open file b in directory pat2 and read its contents into a JSON object
    f = open(pat2 + b, 'r')
    x2 = json.loads(f.read())
    # Extract the part candidates dictionary from the JSON object
    new = x2['part_candidates'][0]
    f.close()

    # Initialize variables for distance calculation
    const = 0
    avg = 0

    # Iterate over all body parts (25 in total)
    for i in range(25):
        # Check if both the original and new dictionaries have entries for this body part
        if ((orign[str(i)] != [] and len(orign[str(i)]) >= 2) and (new[str(i)] != [] and len(new[str(i)]) >= 2)):
            # Calculate the Euclidean distance between the two body part locations
            const += sqrt((orign[str(i)][0] - new[str(i)][0]) ** 2 + (orign[str(i)][1] - new[str(i)][1]) ** 2)
            # Increment the counter for valid body parts
            avg = avg + 1

    # Calculate the average distance between corresponding body parts
    const = const / avg
    return const




# loop over each directory containing exercise data
for c in range(len(y)):
  
  # create distance matrix for each file in the current directory
  dist_mat = np.zeros((len(x), len(y[c])))
  for i in range(len(x)):
        for j in range(len(y[c])):
            print(x[i],y[c][j],pats[c]+'/')
            dist_mat[i, j]=subf(x[i],y[c][j],pats[c]+'/')
  
  # find the shortest path through the distance matrix
  path, cost_mat = dp(dist_mat)
  
  # append the cost and path to their respective lists
  cls.append(cost_mat[len(cost_mat)-1][len(cost_mat[0])-1])
  paths.append(path)

# determine which directory had the closest match and set variables accordingly
if(cls.index(min(cls)) ==0):
  pindex=0

  mean_diff = 0.051979695982485974
  std_diff = 0.04759577403012386
  mean_diffx = [0.005988240967722448, -0.07911618013546035, -0.0741078981868293, 0.0605911913667703, 0.01878972600468171, 0.018778414658363093, 0.015184733866862182, 0.013530343190940118, 0.011581041554664676, 0.0820686515637278, 0.07930144784983034, 0.07779572770589718, 0.16911929449426022, -0.16996400931757902, 0.05703688974218239, 0.06819870075181543, 0.08256668352384333, 0.09436765788993867, 0.0909774018981743, 0.04654046382874549, 0.02789877440956797, 0.006852102276661408, 0.008635736789973574, 0.0059151746134867445, 0.01249789225168148, 0.004864554544305483, 0.015283044210115802, 0.006294060833741868, 0.011319502051249606, 0.007262230174852098, 0.003263213846662675, -0.01207453295224474, -0.002041941201840429, 0.0011883391669134447, -0.0015775376739740898, 0.0008738791955017372, 0.03563772869667169, 0.07703428965151433, 0.03666508063867695, -0.10935158481906726, 0.054641794916834366, 0.05742629488420045, -0.016791163673193568, -0.01580061404936942, -0.017179914581873262, -0.000278454254741492, 0.0007402064409464812, 0.0015396837045310934, 0.0012495261322318292, 0.004645309234946574, 0.004200533175535283, 0.008555817032287433, 0.008052662759739784, 0.007842712025986623, 0.007634479602245284, 0.005451640623375831, 0.0028439495067249616, 0.008614271661680876, 0.006150653155453279, 0.00606299721972274, 0.008050100554760974, 0.005088200237578674, 0.004431374555546728, 0.0018433080490490791, -0.0011530161526943094, -0.0014922210291223531, -0.0016005236078957413, 0.001144051491093291, -0.001049219609265542, -0.0023255418211332386, 0.001479617452197911, -0.00021254226109564483, 0.000524765023887756, 0.0024760557087267626, -0.002389038528646151, -0.0017549916692273554, 0.0011831038820124477, 0.003544211665214296, 0.002975913713892321, 0.004466262018745123, 0.0046906987828567994, 0.0020517129855175493, 0.0010678244905586009, 0.002697377311846744, 0.0026853959680845025, 0.0016547565977795841, 0.0012272324923886008, -0.003280407705078374, -0.004525382472224841, -0.0044836266199443295, -0.0036785432624062975, -0.004844241838905147, -0.002297696638518805, -0.005491963265615168, 1.849593876509986e-05, 0.003023995664070826, 0.000482768726360738, -0.00011610641560513141, -5.295412269413935e-05, 0.0022618345780622082, -0.0007743772037990912, -0.003324598298338774, -0.0011299069974648143, 0.0011734867882089001, -0.0022474022049562502, 0.0001375896475083574, 0.0007234993329662387, 0.002777859840351749, 0.0015027330020687837, 0.000478069507990366, 0.0016695395130411639, -0.005655134206900788, -0.0047126660746220385, -0.00246272362136261, -0.001993466594890605, -0.0022812646773891676, -0.0017473652056984779, -0.0006258305555699628, -0.0010063778170807036, -0.001414025646247047, -0.0021998514464480244, -0.0019619662911950107, -0.0005212136881195751, -2.0014985433683962e-05, -0.0005897388143781124, 0.00016863027443340562, 0.002667020607057152, 0.0014548651510722821, 0.002874057922812102, 0.0008182108626170335, -0.00029221245796269503, 0.01758162953860285, 0.0038576391220945007, 0.002992876949839146, 0.0033231763572775896, 0.0013669194293319331, 0.0034264640101338306, 0.0021060029404748742, 0.0028599424231696285, 0.0024938934214387893, 0.0006579258771727681, 0.005705167363323022, 0.004593780252798114, -0.0022607038338595935, 0.0009223847119021119, 0.04419761929413708, 0.0717873824919289, -0.11202668014765854, 0.015415821627089567, -0.007255041752747025, 0.0733614764718383, 0.02685515460264929, 0.07105552708121281, 4.686919480340918e-05, -0.0009028603009169495, 0.002072496026088714, 0.006073991711257138, 0.008696957878879785, 0.00039423496871241663, 0.008512043985931783, 0.014559064313764298, 0.001380615870602108, 0.007526278051569453, -0.010096761185407886, -0.0035712685792883394, 0.008978015510750332, 0.006415543750373357, 0.0038681126522731217, 0.00414806623546405, 0.004386761431282416, -0.0019181599904487456, -0.006119030679505791, -0.004640504759122956, -0.006316238848254362, -0.006408392742154495, -0.005707898435806068, -0.004740254488339246, -0.00045002398816927413, -0.0040305339998949145, -0.004035799410844741, -0.00406874494897818, -0.008328243820165605, -0.00832848604232282, -0.004452577516656611, -0.0030406607330229108, -0.003971408641300436, -0.009687204171946167, -0.007811299604869711, -0.008807278472418363, -0.003975500825774732, -0.01832521510801025, -0.0049536793054846505, -0.007590053166200379, -0.010020532852374104, -0.005812091247806168, -0.021503145223779115, -0.012899152386093168, -0.014956350316981427, -0.016427368686106458, -0.017690480072856507, -0.018376819967621513, 0.06894620150471971, 0.055640403412506846, 0.06725552323939764, -0.017640405171538708, 0.013645655921860374, -0.01885197824047291, -0.016648853634886756, 0.00026762973077179287, -0.0008363396712019775, 0.0026557298416282327, -0.0012470238897851226, 0.0014293723218143162, 0.003913846019810312, 0.0022352269735504866, 0.002681730093184098, 0.001670580872699495, 0.0002594697916453077, -0.0005162615706839891, 0.0015044658008465362, 0.0019917926879615406, 0.002125908243050708, 0.0015374383961417365, 0.0022338322414019716, 0.0022867774691030813, -0.0028196550840282526, -0.0044374073396496105, -0.004440054682107053, -0.0021541139705839717, -0.001908046269195589, -0.002076165639677697, 0.003365759147878003, 0.0024158126567577157, 0.0023373226881952557, 0.0018505018228539372, 0.0017722215466429879, 0.0024331045344122694, 0.0013747856895765748, 0.0022135223044700684, 0.00209036768592457, -0.009170540861118768, -0.0346509119819224, -0.02303222078824035, -0.10043186333269469, 0.003296505244957716, 0.0010867036913290624, 0.0032051556179435283, -0.012434043946948484, -0.010509457425638566, -0.009504824612345674, -0.017110602959110533, -0.01745948119982877, -0.012609645428919465, -0.021420668798980455, -0.012979543102600615, -0.010780057715080883, -0.00805174953547385, -0.014003635159656087, -0.011094593045283424, -0.010032187408111782, -0.010879012075944727, -0.004294410992880097, 0.0008351739678974667, 0.0074041558706825624, 0.02081768301017135, 0.03264404459941622, -0.0064886785984738585, 0.004350088124844583, 0.003065585518288335, 0.0030406725469646146, 0.002043791198011373, 0.003255402784977229, 0.003006706704087961, -0.0008303277956243111, -0.0029427964583178907, -0.011835486731293945, -0.014088521236519196, -0.008748790636185418, -0.013606201785349548, -0.021215436928494872, -0.013635678652921675, -0.029446986815479047, -0.03141814860245796, -0.003785841200361135, -0.005480205914881715, -0.0025910179702692783, 0.014954819525045064, 0.012115035667915337, -0.006042606639675322, -0.00014573078234266456, -0.008848161372184702, -0.002091822271567508, 0.00025049759836134666, -0.0011245437120826008, -0.0034193812341098096, -0.005952312640539961, -0.0053618232273643564, 0.005920239532624954, -0.044502780399695435, -0.04806079642158723, -0.04497569924493498, -0.035410930330371244, -0.05651052089870411, 0.012863795681280385, -0.007618353936018876, -0.00493910085153429, 0.0008224101405517564, 0.0005156412903783708, 0.010139266090625827, -0.004451399864895833, -0.006837940666515549, -0.005148124203780602, -0.004297784590123758, -0.0017152714086968036, -0.0010774593043714974, -0.006950684746413229, 0.0008011571284867524, 0.0003687919069079082, 0.003494299534420356, 0.0038050758535653657, 0.008997580216761891, 0.004781452509674717, 0.0005398915193678799, -0.0004219028443634537, 0.003927331866301867, 0.011116236356773342, 0.004148308190794337, 0.0053268999094982835, 0.002451397784076762, -0.002394684244829806, -0.003391607498640202, -0.006874099748534639, -0.08164639538295716, -0.005752205439179059, 0.0005705625629617928, -0.0020276341140388933, 0.0030420135565674016, 0.001622305712728591, 0.001760347743538473, 0.0037520329777694724, 0.002927856721293534, 0.0021826738988072114, 0.0008828971735455415, -0.017493894517698075, 0.0345076144266201, -0.0006978368215238478]
  mean_diffy = [-0.003637477382491243, 0.00867161559267739, 0.007330361331895482, -0.03308046213576996, -0.02069863189640647, -0.021636140850047683, -0.02067414528912993, -0.017987232657392306, -0.013040346248302992, 0.01657186769883042, 0.02211443607394064, 0.11728658395181445, 0.23768599255790343, -0.2963859117857543, 0.07587066834179479, 0.15514892523146645, 0.2022639352558722, 0.20778828346948688, 0.19121655398446427, 0.10688505989389915, 0.07515535413528979, 0.03088966539480628, 0.02598388499660199, 0.01832128950931219, -0.00018499571974323975, -0.024637585685771627, -0.0014910654792407285, -0.0004200317510279719, 0.0032393728110277397, 0.014402225856435224, 0.00959018101397078, -0.020457473976017746, 0.0023499923055376602, -0.022624988058426023, -0.0024014440324015853, -0.0089321498283415, 0.08661416431548898, 0.13289330826730672, 0.034715342864207896, -0.14351252455506547, -0.02250163396778603, -0.019227372879093697, -0.02640584161834019, -0.045740832060768295, -0.03127676806293688, -0.011683166828547873, -0.023992763075574172, -0.007118635337505433, -0.009367334389731384, -0.003534812217987444, -0.006498342475506002, -0.00012939236204466574, -0.002701372340677206, -0.004718960814344012, -0.0017157325207892533, -0.006726379117150997, -0.007694075664062825, -0.014373104032044202, -0.0126941431557192, -0.012480711544467277, -0.003744657502009348, 0.0036886721047247554, -0.0034516195529349764, -0.006883135726971698, -0.008546988723728168, -0.02435734732189909, -0.043552404350585314, -0.03230660426193372, -0.03142903838372835, -0.029248042467752828, -0.025335255090087622, -0.00745319062863004, -0.00878453196429924, -0.015447474518230107, -0.0061699787795399304, -0.011972669365192075, -0.007831721987778307, -0.008518467873912196, -0.0042497305471948625, -0.0015390604209221082, -0.006966547323831936, -0.013517733684589209, -0.012384209840391307, -0.005379307098718106, -0.007130075899367427, -0.0010842052574981122, -0.008668495786739732, -0.011470263312232219, -0.001573209802680542, -0.0018196613978731149, 0.0033454423437056955, 0.008271638998274142, -0.002250786188786173, -0.07185824540652731, -0.007236386479596555, -0.00021454658778077146, -9.869785358357322e-05, 0.0015540051395521624, 0.0028612161478980256, -1.7889695201659116e-05, -0.004127007055178255, 0.00018926347362139634, -0.007938439801679803, 0.005134291069922463, -0.0022228204159049354, -0.00461225601857221, -0.0029586088664612232, -0.0018487443807559518, -0.009050110772653426, -0.016818280847035662, -0.004329586187899054, 0.011283762930885503, 0.0036808930943807204, -0.0026993473924955607, 0.009042964840925084, 0.009117173298750449, 0.018822367849310982, 0.0423232107784686, 0.05902420814934018, 0.06786065047095244, 0.0752125483072777, 0.07563158524589478, 0.07466385139873884, 0.05431542250494835, 0.04465833120833513, 0.0349077443085495, 0.031011928374098358, 0.020022930849107005, 0.01709452169400085, -0.0038341569529760566, -0.006413759501272661, -0.02930437762366375, 0.002359227948472251, 0.005638781328463548, 0.007547030516798245, 0.0052179029303027905, 0.004269667977976271, -0.002472793022222504, -0.008320147486388674, -0.017856871473336876, -0.013792563981368501, -0.002463608723402915, -0.0017669458606294753, -0.018770417429894805, 0.0005217227111583794, 0.07477510326362839, 0.10358755374706792, -0.1887303502252909, 0.025713609714868774, -0.004855536898444366, 0.14634259419085172, 0.09053686164311191, 0.13765813559164802, 0.0030411312780287817, -0.01023090729997784, 0.006775787936540879, -0.0018845627091413674, -0.03662621391284976, -0.011942969057462189, 0.003604254445765195, 0.005065183715420851, -0.020477517356316376, -0.013073013088041851, -0.040076000066651296, -0.02969125993642036, -0.02162148328970056, -0.011170631147792553, -0.008393663668388842, -0.006766055713161036, -0.009978077103756153, -0.010530184045148711, 0.000670981760760263, 0.00776930969917574, 0.005662042354350736, -0.0002522499164158634, -0.0013680767231743578, 0.00464070301777392, -1.8615681486576498e-05, -0.004964247605021584, 0.00023191863565162147, -0.0017493291599397088, -0.0036430167852943597, -0.004168348680405234, -0.00030830177401101194, 0.002373401146921403, 0.0027335173662911723, -0.019475191949138396, -0.010690114973244876, 0.00016230213559050258, -0.007074639004603673, -0.033211586545409316, -0.003670202076713755, -0.011480890730372792, -0.007044089493388373, -0.021203711649188425, -0.029510675237628833, -0.015613184025502077, -0.02877791883151475, -0.025736549604365458, -0.03713490070450564, -0.03254594021447327, -0.012056147063688261, -0.026185284133670193, -0.023751863485948155, -0.1112453306404713, -0.02232578353829491, -0.05595396252787738, -0.10595445244405206, -0.04788605438439872, -0.023347646644896837, -0.03085160697102313, -0.004770529589080832, 0.0034943979366992123, 0.010518416416224356, 0.0007096255968644941, -0.012367453314693098, -0.022236917590481572, -0.023140142685496783, -0.020059057303707176, -0.012136502567936403, -0.0060453071172826365, -0.008198667485240166, 0.009702386463720266, 0.013511094735085931, 0.013556227797758837, 0.024649765462227684, 0.027363735334037485, 0.027478740793253457, 0.02230489249709109, 0.02219174513254647, 0.022191914561789264, 0.008127214024860436, 0.004795408293607087, 0.004873429475840918, -0.01214494608426826, -0.012189076505299884, -0.012500334991819781, -0.009476285013437995, -0.007632980492022181, -0.008211506704944874, -0.027646619839459038, 0.034359321731210306, 0.003122063312311151, 0.10849122619643468, -0.008039627363463718, -0.009260206682192684, -0.008970273387945323, -0.02399837761490131, -0.03129478734888876, -0.016603624391107966, -0.029411414358568297, -0.029731345545590003, -0.020152160155460223, -0.0365754503436942, -0.025016816233715926, -0.0177328235926353, -0.008388624726443887, -0.021829682032141577, -0.0348430175812777, -0.03576050746198509, -0.024719657282407943, -0.012952978129257522, -0.0014188536136769637, -0.015037991628831926, -0.02112083517040634, -0.028541442786431944, -0.0001299767515091016, -0.00436425069404667, -0.005269220531006209, -0.0038767680769525, -0.009903827931381408, -0.0077261607757916125, -0.004416382066564808, 0.0013664434524579261, 0.0020759682111068404, -0.016465022832431685, -0.015312872928608987, 0.0016314311579782436, -0.006932125513448467, -0.015036101133598016, -0.0012656997668201709, -0.015388304815386673, -0.02782594351676819, -0.06325427187511633, -0.04237989286326574, -0.03266418589076688, -0.056782609053568224, -0.03695415211965196, -0.023441341491774903, -0.04603657070381118, -0.045389980685564255, -0.029999486491117503, -0.01636430911644175, -0.007124463297100597, 0.00480769580413594, -0.0004858120169501368, -0.014427777897325036, -0.033839144307694385, -0.02675072231636049, 0.010397432910936493, 0.018949141915007663, 0.031147782774742488, 0.07762637067558126, 0.03664431027041636, 0.05052172796438664, 0.02956765791661939, 0.06054962846584842, 0.0732785939105782, -0.010093809598736318, 0.08001007457742229, 0.08496745989046214, 0.07989259768740162, 0.04310062264039423, 0.014956312256151474, 0.027498080933173786, 0.02642586481968402, 0.013497417687768312, -0.006932841024821781, -0.008321822419949892, -0.006883219066968333, -0.007804934625643825, -0.013402589216782517, -0.002906315671734334, -0.0008529677186665873, -0.006601515254640326, -0.015908351731452747, -0.018132794248184897, -0.0007591960339111265, -0.010645132361969723, -0.026444188870814064, -0.015892507073617158, -0.01017945331629726, -0.005347108980459576, -0.014573781406017566, -0.0051439721167964745, -0.004787593671431573, 0.00607032781803313, 0.006126181023766235, 0.014493496872256933, 0.009290041099984138, 0.01392012112295146, 0.014400323944909555, 0.004741777409156325, -0.06476682285432413, 0.022074656418089117, 0.03782356221345781]
  std_diffy = [0.03545691635578407, 0.009911102618797022, 0.05615626074049499, 0.04993210967107028, 0.02733498438336064, 0.027661493998474274, 0.06516503783283105, 0.0675484191085587, 0.07163651134964914, 0.0974386419426124, 0.09530419781003112, 0.13184396572086785, 0.25788282925085787, 0.3575200215431838, 0.09575787102374538, 0.1717458933763607, 0.1863498397095043, 0.18801242547126998, 0.17446170896265592, 0.15983060634376625, 0.1352016030691191, 0.07822256793211158, 0.08174226239985195, 0.07048184399530134, 0.05406626842899805, 0.06669979377603552, 0.0487465744475676, 0.028155617636241567, 0.02942484277174085, 0.01585411719760153, 0.017824419533594867, 0.039565697889836694, 0.012306270716468769, 0.03089420566485365, 0.022388685150322187, 0.014679460953573337, 0.11296231538652261, 0.1255480519163119, 0.09139053195171344, 0.19439311110790622, 0.019502062669550773, 0.01802064087253996, 0.04402426268622598, 0.07297546210986837, 0.03598633180943658, 0.022562257865689085, 0.03552899636886178, 0.02097923456794106, 0.016733409444209847, 0.009359427272874806, 0.015452102951064531, 0.012528351867868269, 0.011115601048220394, 0.011255267817373241, 0.011235920961791475, 0.01755179743587063, 0.016277548288351835, 0.027310419116117195, 0.023021753302993248, 0.02245788383897456, 0.017099686602636135, 0.015322699311082313, 0.01562508939549167, 0.020925648855786325, 0.020744133490903018, 0.03421242749785198, 0.05370988637037629, 0.04321353262630455, 0.04330221851759176, 0.04092899907436658, 0.03587787692837706, 0.022091724257832665, 0.028356714838459834, 0.028547208498467255, 0.031887974289348545, 0.028908497448674995, 0.02421777356542089, 0.03827230890655348, 0.027684196094943686, 0.021539817566158265, 0.03406287833286551, 0.043373831891362354, 0.0361640038361936, 0.022047321185402636, 0.028098675977732007, 0.02609812579574205, 0.027618907980977547, 0.02200352243485354, 0.020864669428024396, 0.021063030528900344, 0.021269486460116067, 0.025892084165967298, 0.02903503149637416, 0.09003530301226609, 0.03262282800337479, 0.0327503651448835, 0.025515933727290004, 0.025959400090634025, 0.024416197875170388, 0.023290905641708687, 0.014949725741397746, 0.024180758673212954, 0.02551545275954223, 0.023672826741043502, 0.036062736314619606, 0.03821127335019608, 0.03911706421712258, 0.04443558684811571, 0.04514489338938347, 0.06302498618363908, 0.03839453426628098, 0.05691235029951576, 0.03408080137620024, 0.031164880159553075, 0.04625299379136458, 0.04635440501468189, 0.05501860210206221, 0.07826228086762703, 0.0930705645999244, 0.09790240724876759, 0.10241148366814341, 0.1032103228992639, 0.10244543824380854, 0.09415561373618284, 0.08601458305408645, 0.08083337334660855, 0.060424149407148346, 0.04949132059253529, 0.04072966743719037, 0.018686445772858846, 0.014406991086696103, 0.033068163041001186, 0.029320927001730734, 0.03564385832920831, 0.0422593399856042, 0.0348513704416276, 0.03615816730398451, 0.02707227220813749, 0.019655461906711934, 0.035546363403208596, 0.02834257162201505, 0.015841783417766687, 0.016241740917087508, 0.029725085329291558, 0.016343639559907502, 0.09650140502253492, 0.1050149664915738, 0.24410292096439437, 0.08696298371973443, 0.050476891819239735, 0.12387703125217867, 0.11936446759902458, 0.12788376148254224, 0.01568243823234066, 0.022796037943067466, 0.011551425092842063, 0.024346443739551686, 0.04045940047022781, 0.01646160871972771, 0.008829861752709854, 0.018078756250890626, 0.02455768683628957, 0.027801317390354206, 0.04813227301686295, 0.03969204244432826, 0.0354280899116276, 0.024298726961955137, 0.01731698288492405, 0.022260100034058993, 0.020271282317629133, 0.023372689614817147, 0.007935208526554556, 0.01035750479647771, 0.0072648975986050295, 0.013972478645848765, 0.013539810539308465, 0.010395626429642788, 0.01403874034221466, 0.01772464540935416, 0.024530630177804724, 0.020861305571675842, 0.018444620700822055, 0.04074137353555891, 0.012139858851854157, 0.011079744089213542, 0.012653704080368368, 0.029154994596294174, 0.02083697114017729, 0.013227601176414942, 0.022346129360533873, 0.04286651619150289, 0.02672722665447902, 0.022550139133741466, 0.026513906077520404, 0.038101765619008306, 0.03545317361006693, 0.03661196730234367, 0.04449144615852088, 0.05395236089282363, 0.062456413283736634, 0.05924835358404647, 0.014162190875078752, 0.02291917502469901, 0.027077963411237152, 0.148032765549957, 0.05814172692372714, 0.07432792373579446, 0.1265604121896303, 0.07878219185220807, 0.040617199697714476, 0.06410811681815123, 0.053158594521562456, 0.0352900351160875, 0.026190856624239988, 0.02960362353415662, 0.0268667944113808, 0.02992480099760453, 0.03088055962414221, 0.029318831847804653, 0.03114145404450354, 0.029593747846915874, 0.020009646558057235, 0.023278132190684347, 0.03406605660033842, 0.03403693698461675, 0.037595783090914096, 0.037921688420921265, 0.03818127173046847, 0.03995154803858436, 0.040195812831747106, 0.04000609732175089, 0.03398659982422011, 0.02807629828352839, 0.02792882283570663, 0.02310367004978266, 0.023036728355456837, 0.0232187940052769, 0.019730822955127035, 0.02293223952736082, 0.021946650008911925, 0.034497443040468326, 0.0751279172734743, 0.01511868381186596, 0.1380633833636572, 0.018737496053979463, 0.017855584796227234, 0.017106791566636775, 0.04514324347827888, 0.042900742501574066, 0.039924525144435806, 0.04521327650174227, 0.04619454306460956, 0.040516545791877745, 0.05324487144621643, 0.043494584936967826, 0.04173844742279604, 0.03102754643611913, 0.0493770011482514, 0.045117578308395564, 0.05359322593477291, 0.0439167759766548, 0.027087821242818246, 0.007777817188373434, 0.027579209792312334, 0.03353566456382744, 0.03579033717028353, 0.01579179243657361, 0.013572304079722384, 0.013305365222973055, 0.01654859243898462, 0.011366292219445209, 0.011438553410313253, 0.015283832031687225, 0.015022008932431193, 0.006552432983855859, 0.02892246351146547, 0.040143834047209496, 0.027899604217553924, 0.031938502705442684, 0.03252825876479536, 0.02056738665428598, 0.03483284265357746, 0.04939704121807029, 0.09936680193939709, 0.06204148471845626, 0.040321789740598815, 0.07296473420060363, 0.05644656178234189, 0.03872685209147302, 0.0723302685377092, 0.06558301195173133, 0.0858721201984938, 0.1115301452063549, 0.08962382387111098, 0.08937750248533886, 0.08021472794611706, 0.06654650241106812, 0.049974166998856324, 0.0649033118971306, 0.07013763934439769, 0.045951890282580204, 0.07710625211382213, 0.13344296684389248, 0.07806851508634481, 0.0708657639204167, 0.043405522990180676, 0.07795469245240615, 0.08862189714919617, 0.04888795932169562, 0.09903546519540767, 0.10682393513500038, 0.10260311486515755, 0.06516201554855308, 0.028910951741638612, 0.04993234451739362, 0.05685361276167899, 0.03803762178764051, 0.04093317259813905, 0.04054882346570166, 0.010817207560698264, 0.019222168817036776, 0.028491327525813834, 0.014252722569963657, 0.018652283427029615, 0.024478880611103832, 0.03345846187448884, 0.03291463922187475, 0.017047915474782772, 0.02682160799322658, 0.03896006839013006, 0.02585907382736192, 0.023665715021080076, 0.019234363139165735, 0.03278064466646136, 0.02652466525999409, 0.020299460219749304, 0.027565001896674952, 0.022336463758427978, 0.018526699852587317, 0.01885832433630486, 0.019589702349103434, 0.02735341858207212, 0.01004716701654064, 0.0836116923859182, 0.06593148048785953, 0.04422428272060217]
  std_diffx = [0.020935882116559768, 0.019518456559049054, 0.03220898787358711, 0.03943195215149245, 0.018236669058993352, 0.017846661304341507, 0.047593561961801534, 0.05181546025064719, 0.06917754941776204, 0.11244927936189444, 0.10823228737916381, 0.1137814152152387, 0.18459092505238828, 0.20645557205561899, 0.09023704157700439, 0.1007082391840304, 0.099566318499959, 0.10508028009356887, 0.10098699442931704, 0.11507569305405535, 0.10940635327156652, 0.08465809260767783, 0.08960730877351422, 0.07429019663592387, 0.05091057506361605, 0.05833720932690858, 0.04497488046995703, 0.028511628475964727, 0.03633990873640646, 0.028989326346515614, 0.03039769175106251, 0.03524532328929588, 0.01623751836331474, 0.012110680436106913, 0.027968179457284324, 0.011323214219563464, 0.06477297053429666, 0.08134332850859534, 0.08673506944707965, 0.14167431890919435, 0.08231711995389678, 0.08102092518793759, 0.029968057753348414, 0.04377783393684678, 0.016445730067710017, 0.009617238486591245, 0.023584057961294737, 0.012888975852453539, 0.007942896390855068, 0.0075650579778005125, 0.00983238656920691, 0.017972546554670395, 0.017448027356561733, 0.017339639128231516, 0.018120678075742445, 0.017320769481683713, 0.010354538107139854, 0.016680351425083652, 0.007160657603286455, 0.0070409708859522835, 0.02482987637333406, 0.020971966206358154, 0.01662103093686922, 0.01878444552288019, 0.01797189667004307, 0.027400959489673977, 0.03632381907350049, 0.031137001011139205, 0.03366093700143998, 0.03186218453635497, 0.03285638203008407, 0.02038956685656822, 0.024418452342267118, 0.021842972253292747, 0.028832783383740263, 0.019658533699967076, 0.016640503489556716, 0.027114862185796815, 0.023891748911706343, 0.02391551539126414, 0.024657962434169654, 0.025958695685771888, 0.02432624686355261, 0.022714493408770478, 0.029208806471817117, 0.033495135020759466, 0.02542685468632049, 0.022962387962601808, 0.027970964118525822, 0.028000119830850597, 0.02292729666031691, 0.022027817883133616, 0.021286993475845124, 0.06575911275400442, 0.021229875573511497, 0.02234774725532399, 0.02719200944338264, 0.027745454579842354, 0.02730577347028245, 0.027550060297916026, 0.02242873782235838, 0.029490752220623927, 0.020495335698578792, 0.028127944273644682, 0.024383445188048206, 0.028843684149701523, 0.02901157747426852, 0.03210989014278603, 0.02396599491296092, 0.03744927403040644, 0.02549076198944849, 0.056934484301402195, 0.028722786698426216, 0.02005921602083381, 0.04912199263362621, 0.04978774242703875, 0.052501784500145136, 0.05647642497999361, 0.059547105832480844, 0.06262949941452331, 0.06919033212072209, 0.07233312331402121, 0.07354684073340684, 0.06425536982106712, 0.054687157377758436, 0.05105064489847099, 0.03761922996750674, 0.031909151212860316, 0.029449012207289835, 0.01839573521321013, 0.018518571554408794, 0.024561998352572195, 0.02228595310616102, 0.02664040127305436, 0.02996526367105576, 0.02485609326074201, 0.026784708545478948, 0.02263355524025513, 0.023326612656733976, 0.024714245653784696, 0.022190803803362823, 0.017009664528881717, 0.017376162860936965, 0.022693730457994096, 0.009039862984114286, 0.07630118818978215, 0.08757027787036184, 0.1506004570273265, 0.07752387709408477, 0.029349187098703943, 0.08327600855289152, 0.06921971217745602, 0.07719685472949026, 0.02558545410496572, 0.021157952396576904, 0.023318854141375828, 0.0189325486034467, 0.019853752952872873, 0.017486075045240336, 0.02139404187312124, 0.02946946724161287, 0.014319207950492104, 0.013328546767636647, 0.022295478288284116, 0.0242248826156695, 0.019658634541174204, 0.03064749346406089, 0.02341173781618131, 0.02852607055051278, 0.02249982266580487, 0.02399850008873855, 0.009093324878929486, 0.008426463584430131, 0.009376015966872368, 0.017148155362794803, 0.013260134480642413, 0.023811694801311192, 0.02134044916107146, 0.019134516893859598, 0.019178993042716207, 0.0074318661859683385, 0.008240245690952636, 0.019123780695619166, 0.010686321503718968, 0.017642379186341463, 0.01659301752801746, 0.009446274312560714, 0.010643588038006841, 0.01277086343900649, 0.01888343820969634, 0.02351175074840853, 0.021760042904836106, 0.021544380800815627, 0.023502476298577223, 0.026574631134182656, 0.02461150212447568, 0.028618573945244613, 0.03302096485312514, 0.03111086161374969, 0.03142552284174594, 0.026628455890430697, 0.09827475457057408, 0.09880695806059088, 0.10692702210403346, 0.12792095812271667, 0.09541748930400802, 0.05351890974205196, 0.1166182823959863, 0.07653098920545649, 0.03165672240286359, 0.06158556830879322, 0.030276997481718138, 0.025667825645978325, 0.028490725415326785, 0.02908396968725331, 0.023131889650711626, 0.012946882628052358, 0.010099852213246564, 0.01045780379816789, 0.02230728122556409, 0.024350195080931916, 0.016426446624194898, 0.025115328223255483, 0.029243254668890013, 0.029244508380123175, 0.03532964328422623, 0.039524945025309115, 0.03966424688478578, 0.03599857456639806, 0.03615380826147367, 0.035909339466543067, 0.025224541065373817, 0.019404120016288585, 0.01931247515198288, 0.013261442997804941, 0.012698705331480273, 0.01613265514857357, 0.014220771777292139, 0.013242722800236867, 0.012676195576997312, 0.022685294886537626, 0.04476348594999594, 0.040607434559008095, 0.0575804488182691, 0.018927079239412243, 0.009227403074643481, 0.014794811685850345, 0.03241237306595097, 0.025857264238208066, 0.028975106233611686, 0.029640007503554607, 0.03107647297185059, 0.027051253374055183, 0.033092810841791886, 0.025826515667127046, 0.026561703414328907, 0.019804185803695182, 0.02784979943992252, 0.029143749999541585, 0.0305389502575479, 0.027507916728147033, 0.019775745988546298, 0.012662152852594056, 0.026356588642822452, 0.03818139932472471, 0.034772519014856874, 0.07985154331201721, 0.015663759963036233, 0.00852950529281875, 0.008876571154985491, 0.008235471468828014, 0.011298892783618142, 0.0176150283156362, 0.01940149528867112, 0.010363406974202674, 0.021033456479350468, 0.025698480204681528, 0.03114834008552179, 0.032737773063393566, 0.02440361976871776, 0.019797695010615332, 0.06831622141847946, 0.06840184547670024, 0.06401588501644818, 0.05121373141494109, 0.04693075513505602, 0.11378568026359276, 0.09672064352367453, 0.027042621104707218, 0.0526466639668076, 0.04182615765033264, 0.0685202699925248, 0.0931178065102027, 0.06682228952493492, 0.06379846404981747, 0.054018246587422915, 0.045253861624296426, 0.031556368496764645, 0.0554645317032032, 0.055846371248077946, 0.057959022625262285, 0.06834397089175363, 0.07739416361502569, 0.07093173902147693, 0.0637611233751823, 0.04783661931838533, 0.08427129780099275, 0.09893311206572225, 0.03435114235801361, 0.11146082779538502, 0.1060352609916386, 0.09983689890912513, 0.058131888711958964, 0.028173399002260045, 0.039848855076074496, 0.06551058059858107, 0.03030871556693805, 0.042815108600012586, 0.042600678304445035, 0.031725751109016785, 0.020893519649219303, 0.029752802808085614, 0.012052910173816931, 0.014461897732745204, 0.022744987004434853, 0.0231936195095007, 0.026960285472438356, 0.02705766172610598, 0.019989286870151318, 0.030202925199827473, 0.024398500533152366, 0.022033595044206038, 0.08384620791779435, 0.025315625651880762, 0.01667352379225258, 0.02137189091763443, 0.021013935593181807, 0.02064578725725426, 0.02626927267998629, 0.01974245043644439, 0.021948176841048782, 0.025277028593172404, 0.007806421604518452, 0.08835354200845531, 0.14859974348519508, 0.09590871277167082]

  print("==============================================================================")
  print("The first exercise (Jumping Jacks)")
  print("==============================================================================")
elif(cls.index(min(cls)) ==1):
  pindex=1
  mean_diff = 0.05743875668179735
  std_diff = 0.044855065215512704
  mean_diffx = [0.038387251711037434, 0.03377448361951824, 0.02326175378428955, 0.03927890584369684, 0.05216929384498638, 0.055814948370086555, 0.04960622087695362, 0.049972875369732585, 0.05507151770292021, 0.039505241836613074, 0.04892130828718997, 0.04313015330494337, 0.04011115266443281, 0.022233969401656756, 0.03122496500663071, 0.03634627261557651, 0.04787691945415855, 0.07206622201558802, 0.0675538046698717, 0.06393432151614792, 0.06368378441517088, 0.06277701281099729, 0.06149294039677761, 0.06486990660135664, 0.05959021040976962, 0.04392332257232536, 0.062111530919539124, 0.045792045998039425, 0.03674145109527407, 0.04925551731711405, 0.040303394766578314, 0.04825135938245301, 0.04020134320410371, 0.020164989849351177, 0.10912812015415373, -0.2653254079258546, -0.016879605096759946, -0.03224979445884644, -0.044311947235585424, -0.028492809226366177, -0.00920334015643657, 0.013485177311799146, -0.002951892894494683, 0.1456122094033184, 0.006802450158162323, 0.20896905029332052, 0.004371973263566128, 0.026543476715739752, 0.02380132831196762, 0.0060520750812418445, -0.0032200268642512547, 0.004625563353709138, -0.00030566937519040967, 0.0009031407983664963, 0.03357299102652606, -0.0011868197351072919, -0.006178233702772648, -0.0007063744660987803, -0.0021812030757850753, -0.0020677626401933015, -0.00012095403901737633, -0.030519072815320215, 0.011004450607768586, 0.03734858869772831, 0.18492059453772533, 0.049082814517186095, 0.01898234891540925, -0.051441522579452344, -0.015401363766749849, -0.005673802116065849, -0.0020059570992785264, 0.0007055548382198105, -0.0002913210909386113, -0.0034407681691650577, -0.003921481882330533, -0.006913800944421432, -0.009367377677580913, -0.009442480685745576, 0.001265820567222527, 0.0005514263707853048, 0.004294836833326033, -0.009192104487659973, -0.012092991357373257, -0.009826845155061804, -0.0069008476427638455, -0.012190546264301405, -0.008382718593519791, -0.00862937357198878, -0.012660404517717922, -0.011541836080338238, -0.013850424880953711, -0.01661035770124422, -0.0200284431353222, -0.010447890967026991, -0.01641915022651348, -0.017167763775994174, -0.016847451261764446, -0.015733049437239122, -0.015111849797542536, -0.01684862300049731, -0.02800161779461086, -0.01823835997043747, -0.007453479059608253, -0.014469943993491836, -0.01659696364261118, -0.016549736693675557, 0.0007482336570524917, -0.01618472808383883, -0.016491759791166048, -0.011081813486299791, -0.010794776283737534, -0.010336514843968256, -0.010346651202749626, -0.010124611017769744, -0.013140353456543685, -0.051723925501133416, -0.010072594659238352, -0.010515725707457008, -0.013794475871762963, -0.012124122703292564, -0.03280434666028032, -0.03210987519454739, -0.013195946827322598, -0.011969582128767291, -0.009817577273918743, -0.040001250631538594, -0.014151034640253933, -0.011183724023942616, -0.01097959068076158, -0.009817998683393486, -0.0103909967063333, -0.011186948698493545, -0.00934439581823388, -0.013149616893056573, -0.007436316096230956, -0.008848446885182078, -0.011185634378435673, -0.012901246891435987, -0.00671216349795433, -0.0011412788546560975, -0.008136047451501363, -0.008574831498028948, -0.037473657835595635, -0.019225610333276643, -0.008766887804725974, -0.033929380334661335, -0.04181231098249064, -0.06819281438086283, 0.02901173813801905, -0.02773483263596726, -0.04051171686623323, -0.05893907774445208, -0.05952703877508379, -0.03697504709989701, -0.035303379251814176, -0.05579341322996509, -0.053475308750024625, -0.05638362550164503, -0.020994993250134052, -0.01915521469602269, 0.009254427322415205, 0.016937052914548247, 0.013863786434279768, -0.02557933480936413, -0.02157987732569465, -0.01734570021891338, -0.024243128708863405, -0.023595037096787827, -0.01662293601389775, -0.006950282700985589, 0.007737750795122, 0.0017028710378735096, -0.009323008205734373, 0.020256656497749312, 0.0016094351074062807, 0.011072751453859657, 0.016094837674557053, 0.031113972395107013, 0.05404621472549295, 0.04291449344281295, 0.031132734529773724, 0.03358865758741425, 0.021422323208877776, 0.019533630587235874, 0.023712973740116973, 0.022639847743030475, 0.026793083994349307, 0.03367955893117005, 0.03217947577178195, 0.026454457542637106, -0.002798908408231871, 0.038806935084775486, -0.0017458157536677176, 0.021809292748105266, 0.029553765069177666, 0.02466334212365254, 0.003353120561993629, 0.017497085079539273, 0.017720522294595763, 0.015092228342189555, 0.02787890400014067, 0.025900951813919274, 0.024647065582673506, 0.025373250736811546, 0.017339490789389983, 0.025915408895089555, 0.027446411052412304, 0.019514163879415566, 0.01680638737594349, 0.018098104698429242, 0.019284338824365276, 0.019937042257152714, 0.01765497512948376, 0.018436102347074712, 0.02012445460359953, 0.0179196737468992, 0.018663630799539074, 0.017495890213324105, 0.01425568618875801, 0.013744039127744547, 0.013061039760574347, 0.012669115789689536, 0.012517316878520014, -0.008084244479810711, -0.007586916076191723, -0.006699442419305237, -0.006168107138273478, -0.006663201938873525, -0.0068409241272402565, -0.005990896848148547, -0.005755861289128207, -0.0003070679441293378, 0.0022527814923246838, 0.005618495882207102, 0.0017054700074305407, -0.011382491962263905, -0.015499385255273179, -0.00798279823119205, -0.0027987760431354048, -0.002453504403560968, 0.007685807280289274, 0.002788790583759343, 0.02882396956943982, 0.03528110794842831, 0.03083152213665886, 0.018443373895626808, 0.018871355874701002, -0.003324021642099776, -0.008974687978452777, -0.01107896730270328, -0.017687157787277096, 0.007119966498262267, 0.029975059772460934, 0.02092421158227968, 0.036737669442472594, 0.04376487200485107, 0.05362937064962928, 0.04816290746128692, 0.05373785685050181, 0.02107328712744437, 0.02508326536080668, 0.025719629010961392, 0.02405492716377931, 0.022719811405502454, 0.022720211972080046, 0.023668907547676074, 0.02441899414512329, 0.02478277083164114, 0.026626108755848678, 0.02661414655739805, 0.023427210256890988, 0.10224282107759708, 0.03565605058705405, 0.03460513427294569, 0.03360430763210465, 0.0277356406392622, 0.024883260124167625, 0.026312631277082527, 0.09358748485962093, 0.10613181075977973, 0.023905655755615395, 0.024218708651828628, 0.02039783060210498, 0.00631856391895382, -0.06402033118682898, -0.009407460358160142, -0.006294832510338098, -0.058944482964114496, -0.009691504833160893, -0.03802937012274246, -0.017770554178855844, -0.01771863381669837, -0.016791072166135002, -0.01105654566995714, -0.015565358449647295, -0.01258703810264523, 0.057105127655038605, -0.0016819492229715695, -0.003063917849900225, 0.00020072015152177268, 0.032374078458665204, 0.002834363279831259, -0.0025941714747213696, -0.01103603518798154, -0.0012526372253117313, 0.002546704368923205, -0.0012497431206012254, 0.007668541759177254, 0.0016715573915065702, 0.0036250754507615923, -0.008166219344605343, -0.011879445965465807, -0.014407802786383806, -0.06896505887764955, -0.07291007226283502, -0.08729058116975928, -0.06569778468682033, -0.0009750971592185695, 0.0014203774272812052, -0.017362958957845752, -0.010145050007704706, 0.005287408983583573, 0.019878985078368844, -0.030771732893330368, 0.023062269506843994, 0.02784383966805428, 0.020756683858269728, 0.04760360670000351, 0.044097992002658515, 0.030465710043291506, 0.07142056828723235, -0.011123739847650908, -0.0032846830763579703, 0.04007979823962634, 0.022854620478062863, 0.045979671482310296, 2.31208699403318e-05, -0.0012924797544304207, -0.01998809664189989, -0.006480612092746974, -0.018598396253517707, 0.006702645980226027, -0.021441106776736305, 0.0006281433634570333, -0.0503041614852847, -0.03243825390986659, 0.00624072173827913, -0.033284881704155944, -0.05532756046207496, -0.07601900688974224, -0.05043336901642556, -0.0706462314566004, -0.04702186195000274, -0.09345033285821885, -0.09992395714888126, -0.09864280689912454, -0.11291742859843915, -0.042367273715014434, -0.017467200302097766, -0.09435654238551494, 0.011155908907209548, 0.006467903504592433, -0.05850950885059051, -0.05578456905307618, -0.01846748620627553, -0.006870632294450008, 2.7990644100880922e-05, -0.002423050262750635, 0.01419372959949329, 0.002145807726104847, -0.0009962246488909034, -0.0014506670979226367, 0.0005000328945906847, -0.09072743922436778, 0.011414783084306497, 0.010398181621248453, -0.0008076980952916274, 0.0033717894354930488, 0.009942673610286355, 0.007951398531602516, -0.004717979287463173, -0.006995611866402193, -0.007574558289021351, -0.007016531112098868, -0.0034732788887613677, -0.0012079596326907496, 0.010041288102210938, 0.009116961141004647, 0.028106986905819583, 0.015292066451164371, 0.011675928412370234, 0.012452121553704365, 0.011067704952163901, 0.015194995691305184, 0.0131314493860602, 0.009913509981458058, 0.010448385201444775, 0.009268373639220805, 0.009716070054956618, 0.009795610734229535, 0.011415328874083808, 0.012344735059298879, 0.010419968725080943, 0.010484217845237467, 0.008905354383334575, 0.010523593454369954, 0.00813308580341657, 0.009347630860847403, 0.009439899711406156, 0.009118229481563087, 0.009126219826383654, 0.008954890020122439, 0.0031372414357377517, 0.009510310014187366, 0.012712301173888822, 0.010898125053993386, 0.00191415731544799, 0.0061160451560042265, 0.00886530915491812, 0.012334728957247845, 0.012378302888477194, 0.010648907514767032, 0.01063408808556233, -0.01668684167734685, -0.004703524451721498, -0.011096380002712418, 0.0008869265923503133, -0.004192775764978145, -0.0004657544349964238, -0.006724376631309334, 0.03414841770081258, -0.005305538974082505, -0.004372050014727874, -0.005634588858859812, -0.0007894992964665142, -0.00932994878595122, -0.03379083717748495, -0.013217648928873275, 0.07943532404252517, -0.046572035428419756, -0.0355374884602309, 0.006313800959722301, 0.007139725380963151, -0.03227163114290848, 0.012887390820174401, 0.014623940900061683, -0.0247396826193519, 0.01949020394889482, -0.016579828225567545, 0.01768149665741435, 0.0054779346957015106, -0.0013428625641577633, -0.003639987032709913, -0.0007350264540015414, 0.00063040504716323, -0.001401816038112084, -0.002868877358752516, 0.006767896694160238, 0.0025119271212988613, -0.035154253769651915, -0.013521595637035451, 0.007433610817332614, 0.0001732026277107318, 0.0002072855148042671, -0.00027872375250920375, -0.0005335695831916082, -0.0012374300000745325, -0.0005741562337074757, -0.002702767326076121, -0.0030253109266392634, -0.0029099253840926924, -0.0027523079668961815, -0.0027400926069424164, -0.0034270522139633057, -0.003781719599622551, -0.003799413976240176, -0.002623507398574798, -0.002073418233709165, -0.0010634438676182883, -0.0015281241534313775, -0.001458219344422642, -0.0007384570818921943, -0.0008161652161565496, -0.0004927271884327351]
  mean_diffy = [-0.01405087376647809, -0.014272201333826473, -0.013873852666802713, -0.015403064356419121, -0.016088587024211516, -0.01857235558875125, -0.016553282665677026, -0.017350658856616408, -0.019107529380106177, -0.02034659031659314, -0.023426035706906358, -0.024740735659808263, -0.017552852108888115, -0.01729090152138129, -0.01714708664947139, -0.020299346757906848, -0.03435149095502493, -0.030491013617307296, -0.03170704953068198, -0.024135211839058646, -0.023177062830197714, -0.022469182492930326, -0.027474612343372035, -0.0334503012011232, -0.03054864521296024, -0.032514586900289924, -0.026673208223324096, -0.024447133333810385, -0.027192622647862083, -0.035556570681036195, -0.02387206837167977, -0.01835881950814714, -0.020915805911369628, -0.08527674544700473, 0.0064982070805061735, -0.15580643225581178, -0.05447540584006278, -0.001511276080290283, -0.0022013062553991967, 0.005286210327037104, 0.009321357120395666, -0.028733770176200796, -0.012570855490804753, -0.05379611406357425, -0.04453226093228797, -0.06448990475415763, 0.003096647837152833, -0.07266901229592192, 0.02374898486661408, 0.041012672868114525, 0.0018670000181950181, 0.0011969428347550688, 0.009867521353387042, 0.008851217011190087, 0.08759978628151385, 0.028057691572785783, 0.021994502342050998, 0.02493498336602056, 0.03268130049521359, 0.0272317462181376, 0.028304990515299527, -0.03288787527226231, -0.006410775637666673, 0.019677116049629895, -0.0024140472627029452, -0.0013945571951779268, -0.009366822756830461, -0.11258676850848208, -0.04100629467646053, -0.017924085652421727, -0.02352619687012431, -0.02429106193957955, -0.021962077625296186, -0.024591553235427686, -0.019255763387419543, -0.018545851425174172, -0.01779363635652593, -0.017447064342055566, -0.020976704554246686, -0.017488932586962973, -0.017028488958919674, -0.001134975633817103, 0.0066886808809855135, 0.011551320130700139, 0.010243448842450147, 0.009783426676212244, 0.008768845258345306, 0.005017434130890908, 0.006415257056479379, 0.0006053944133069819, 0.005192355319979509, 0.007558101975440998, 0.005486079272752204, 0.0032299127099167857, 0.009789857253566028, -0.00017204683673783584, -0.0007140339225130618, -0.0030727789330410884, 0.0009765656028118844, -0.003369294508898435, -0.007072923843133123, -0.0004668263564357822, -0.009104664258785132, 0.0008600410060050276, -0.00184395027875489, -0.0005893234924982499, -0.0117355597365855, -0.006924458825290576, -0.004959431501056356, -0.00028726495242482495, -0.0005465552453646627, -0.01424096009577435, -0.009580338900061613, -0.008455850330941882, -0.007693992533226168, -0.00879740733962295, -0.00494430634978912, -0.012359822994356477, -0.001786298994029774, -0.005066211387201737, -0.008457946780663958, -0.008516802953555017, -0.012914734014447194, -0.0018192157216744215, 0.0016044292317065232, 0.056280866445114396, 0.011547005449315272, 0.008212896500656075, 0.0059740362240174845, -0.005057023699868502, -0.0081563749681673, -0.003297084350376707, 0.0007087516205570305, 0.0018695946940003644, -0.0024457648191687576, -0.004905883372735731, -0.0059923294182418685, 0.004478349543925916, 0.004528325131906092, -0.006090603202799909, -0.009920299935772352, -0.005972200155883606, -0.005108968699348067, -0.000653643649950206, -0.008516426692535568, -0.008536694323228247, -0.0037746375558533863, -0.0035272090798987767, -0.02807849787246123, -0.004804139329612451, -0.010058397886388182, 0.0012915750520708102, 0.00024498132438459884, -0.011303513487553436, -0.008220707834657778, -0.03525806829604766, -0.031077455902235807, -0.03690465161898575, -0.022290663184397857, -0.01447154580772012, -0.012098993571430445, -0.01720075551809936, -0.0193048335796171, -0.018487426370715532, -0.0006992259277820062, -0.0026540816328783023, -0.004460908578568676, -0.009928732829095573, -0.006900802599473563, -0.013485757420987796, -0.017228156039239106, -0.010235726243320784, -0.0064437342852531575, -0.007035432871715659, 0.009031302667891105, -0.009004237978690505, -0.00811329146661342, -0.015287650660413403, -0.028475355654119506, -0.008825624952509213, -0.005228949938908474, -0.00038721415403577484, -0.006604964605540304, -5.278399129527878e-05, -0.006249278203724755, -0.010305161537176305, -0.013996910252956124, -0.009083073502249177, -0.007546146302938808, -0.01698152757766751, -0.032163174498513074, -0.018882641603300755, -0.028617435425548177, -0.018391806490815094, -0.02073288587715311, -0.017551789368776306, -0.011910146851863632, -0.018661132860733117, -0.02201924241025484, -0.017751458421405894, -0.015125010877939377, -0.009437058489647305, -0.015740588156663515, -0.012756777728363644, -0.009091669378148095, -0.014651200582789556, -0.01067157078033125, -0.009218711593297924, -0.010682022781163153, -0.005462211069809499, -0.0038765279802146935, -0.0027665787955403085, -0.0066400683178937745, -0.008075235614437053, -0.005114887792925745, -0.00522522970611579, -0.00451992281699249, -0.005913583674362123, -0.005601720912522731, -0.00433662134599621, -0.002838630431154032, -0.0005155222955673688, -0.0020447060645479233, 0.004364304095599257, 0.0039048539136677675, 0.003800691217649201, 0.004517708576684837, 0.0050590937804531185, 0.003779460016539962, 0.0030734400270454724, 0.0033803326960308066, -0.0060807693202103155, -0.021698872634962743, -0.00516669313969717, -0.004281649512871661, -0.006116848424870115, -0.009722119611525854, -0.008243018595598405, -0.0009263331283553331, 0.017385385918266837, -0.0035228057791384065, -0.002807362695744399, -0.0015172558430302724, 0.0019186122013499702, -0.0013058111094989297, -0.004345734361872586, -0.0032274872180778126, -0.012609471268822767, -0.013799929140663542, -0.010793947249840771, -0.012330744706713768, -0.012313776977889726, -0.005569676224866111, -0.005087545526300813, 0.0025353504513767468, 0.004595212160578826, -0.0037970849700444754, 0.0031155553802658096, -0.004615339907853422, -0.003678044479958107, -0.007596667851083772, -0.006605045707735752, -0.007629361949975084, -0.01225895768354379, -0.01270666725164279, -0.014777479014966724, -0.015936640151136862, -0.012969725595783172, -0.012863504037828145, -0.014901489159398186, -0.01176864929009926, -0.02600910029895158, -0.016348267779634454, -0.017122042261650535, -0.013475703181659441, -0.01608256378952549, -0.016392154859508016, -0.018146733540980307, -0.024136078876550785, -0.04898858891363995, -0.03287021717423738, -0.02444186614037763, -0.03233330029076264, -0.041183804534884724, -0.02300514766099195, -0.031827928546645505, -0.03948226824248078, -0.03760217081010327, -0.05796348530993371, -0.041562950425034904, -0.055676924891482085, -0.04837418889764059, -0.06505084365979572, -0.05257127927095054, -0.059417389513140545, -0.04740316595418306, -0.05740314190580559, -0.04048164537282978, -0.044922573593937115, -0.04958652286673797, 0.01890966315328986, -0.014955663729294837, -0.026992672230357905, -0.013623770254422057, -0.028842931152658992, -0.01923294293986055, -0.016269448209234757, -0.0159655257108736, -0.04644381610150629, -0.03634614220749023, -0.010379163308468053, -0.0069125513098690795, -0.03643307071881577, -0.016160538318768066, 0.001433865082733823, 0.010793492476835422, -0.015541229563126037, -0.05189684759958761, -0.05064325524088769, -0.03185857161120386, -0.02438659310753146, -0.02228614855168608, -0.02196848676474358, -0.0095838605652736, -0.023186498175867123, -0.02610066080200027, -0.021275794016098266, -0.020282687721120247, -0.009270471307920268, -0.02303382654911086, -0.038875967316956264, -0.02026966091894651, -0.02891028028380614, -0.04307390505223019, -0.05258054836027657, -0.06309259718519429, -0.06322310528612865, -0.06350752666620499, -0.06333973415831717, -0.07489602092375003, -0.05144820617869657, -0.08041467244334122, -0.07725220317465849, -0.07857472509824477, -0.059978120370540955, -0.06707179201923562, -0.07641993165202038, -0.06561849157562556, -0.03524596333090341, -0.01862848668345333, -0.004324729287816687, -0.021595621908524435, -0.017171568675556238, 0.01763338014810613, -0.005477471873124587, 0.007857080672815516, 0.025936865050295247, 0.020559896105353852, 0.06249241042204895, 0.019021392483419948, 0.016885326928943952, 0.011784589238540194, 0.03735578211266119, 0.23161896010272753, 0.017982813008339354, 0.00226730180809833, 0.021585772832458536, 0.028942638011068805, 0.011146978712295353, 0.007921553633880777, 0.012654692633821654, 0.02375763560545932, 0.021222567718054277, -0.005546062488425144, 0.009033264324509635, -0.01610258407776916, 0.006940194977389072, 0.005909063755875901, -0.005642502380872938, -0.0018743285450374082, 0.009304624341531991, 0.007811063750708936, 0.0047905354350629105, 0.006045643537684659, 0.0035931252599949305, 0.0008867826235725767, -0.0035094129292956916, -0.0022746521907737456, 0.0024435002799462345, -0.0013817639477925244, -3.0036559734855777e-05, 0.0003525634129223999, 0.0035915282922329263, 0.004999990280209783, 0.003596398906376794, 0.004060799884155929, 0.005945959808581029, 0.005812772165305799, 0.009018666836898153, 0.008268000398891364, 0.006707952864273761, 0.011918604461622979, 0.006295836604328106, 0.007771445743643333, 0.0031032999546058756, 0.004619205536627598, 0.005851997100145785, 0.006856095966936136, 0.008531172225221408, 0.001389151308985374, 0.005328147653368608, 0.008317568522541625, 0.007272961906542241, 0.007399907425468711, 0.0035176876079866747, 0.009816558966138466, 0.006136014620534739, 0.008343265231522741, 0.010322537345967569, 0.004610183063821441, 0.0043119078455676205, 0.003689490192182377, 0.007818535839408374, 0.04538768780169502, 0.0025415640624016655, 0.005722836460974253, 0.008607311269215035, 0.008433972962028942, 0.010087781329552507, 0.0045680380284479, 0.006642772162390367, 0.005657481036020901, 0.008459456423224671, 0.027888095058286072, 0.026547041194302866, 0.022764431182799087, 0.02714529887022764, 0.028916375268493777, -0.06877365671958051, -0.01779534455365709, 0.0011442652976747735, -0.038678444806338995, -0.03471328854895657, -0.06089650014272339, -0.01502107662795457, -0.02627718183816266, -0.015088426760471093, -0.03185471941617123, -0.005590421102004513, -0.023586088697512216, 0.02476765182554328, 0.01453048234362858, 0.016088731705962894, -3.06394400957849e-05, -0.0030058128701477855, 0.007556044446352981, 0.008497270755429841, 0.01126773060953617, 0.005059758646560886, 0.011101444594048115, 0.010391750544385143, -0.004145501258584962, -0.003913399686861219, -0.0027851132780028153, -0.0033545312050537496, -0.006346859760528908, -0.010566370970025208, -0.017956315499149486, -0.01290107830032482, -0.010683854978965282, -0.009566214631627271, -0.00948686425683803, -0.010201729051906923, -0.00801608764339487, -0.011732832587352384, -0.011629560073683633, -0.010219511422390557, -0.0056173511875345595, -0.004280565361582316, -0.0016130546850584948, -0.007536840409494627, -0.00726177742701404, -0.007463741477422226, -0.006187739614466945]
  std_diffy = [0.05988381341317611, 0.061671061564539595, 0.06442769840649595, 0.06456985400417817, 0.06479950283823169, 0.06435013099871079, 0.06499312169798983, 0.06513326621733435, 0.06515388202080882, 0.06176095489510779, 0.05795962211812166, 0.05692622914729841, 0.05836157143228568, 0.05876092481017624, 0.05628273711506648, 0.05637954290343744, 0.0510972924545196, 0.05150244233832521, 0.04991646914513396, 0.049242631292051295, 0.04714092502326468, 0.044563704790115136, 0.04252766735735658, 0.04095373765756153, 0.03865335605999719, 0.03437660922631841, 0.032844605946651145, 0.03387174564004751, 0.029331772970235388, 0.0314738874335153, 0.02598552360318358, 0.023307397636524937, 0.024785112303333603, 0.0612599523295729, 0.07392606805263224, 0.2312987628682356, 0.05095421875182124, 0.015231199036980898, 0.013644131913456287, 0.03114538048446006, 0.03161273014340363, 0.10050934470776664, 0.08904910618987459, 0.16334266848349227, 0.13563864052144858, 0.16360742307169032, 0.07870276548694864, 0.11949813219697117, 0.26259253045880165, 0.04532258865553356, 0.014149859670146644, 0.024903724280179344, 0.01056629523876507, 0.010879553151256124, 0.11197635974031672, 0.0663716350977658, 0.05535904534421807, 0.05790983625520032, 0.06591504426685872, 0.06188495851097409, 0.0807527155292279, 0.05792368895890623, 0.02592117423586252, 0.11233532335323777, 0.19518110600969624, 0.1490043051002002, 0.06943562074225508, 0.2392493838935929, 0.08266470709755218, 0.047925080283271575, 0.05045963236924409, 0.05711697148735251, 0.05320925817382671, 0.047705380013177374, 0.04690015570039303, 0.046090567122959517, 0.034191688743834686, 0.04588699233759841, 0.06374405313611776, 0.03745701192423251, 0.03770164041365621, 0.019203378263647493, 0.014820722469389343, 0.01993522177596743, 0.014785462707967796, 0.02011835716299931, 0.01150153749674686, 0.009053392349578592, 0.013885737245756605, 0.00862810738791431, 0.010905250359003346, 0.02039856889901968, 0.01011628947354971, 0.008648458266873505, 0.022347490058350593, 0.01831625237967701, 0.014843695615717462, 0.012381045471448894, 0.015728983000948122, 0.01628552577862412, 0.021766692627806318, 0.019457726095734748, 0.014708479705910837, 0.02086671677836603, 0.015897497953282557, 0.019717921843933096, 0.01621262256167442, 0.016851520084268577, 0.017958032229249788, 0.02109099011339129, 0.025080905071276544, 0.02341899804826889, 0.020774751458429157, 0.021884756671621253, 0.021312879428618644, 0.02377934099248108, 0.031011199344209258, 0.026588899456918766, 0.027519391083645005, 0.02750845084499475, 0.029896647360945217, 0.031023426814668513, 0.030860257598855457, 0.03432589657038841, 0.038279501913638395, 0.09021704355280609, 0.025298157022431992, 0.014717133129566858, 0.007945947795585707, 0.007167269072942261, 0.006666424667914637, 0.007850070492710196, 0.00871746223613299, 0.007417743229128405, 0.006049588341480977, 0.006825075915782429, 0.008528555950711059, 0.010611919588042041, 0.012422069577860849, 0.010171420501087035, 0.01049111174989757, 0.013903617945944169, 0.013455544675316317, 0.00536112127825799, 0.008631570187302658, 0.013707800357726365, 0.015261232550667247, 0.01632614857324503, 0.04029361753024741, 0.014605579443554718, 0.0254513265152963, 0.03600351023289364, 0.035437027959100365, 0.020463250706528498, 0.020919057762480542, 0.03903109924738359, 0.03179967104629928, 0.03941902203639962, 0.02220699058893603, 0.015178670603300772, 0.03599129646593712, 0.03948655190685748, 0.039628755987647146, 0.02676650464241272, 0.03667835538108332, 0.038413848046372344, 0.03804988967018496, 0.02030948730692212, 0.03376921244515874, 0.03134202055521403, 0.03485255981513166, 0.03076710277596811, 0.019975384555532272, 0.020356119924703417, 0.014054458931834034, 0.015958757334769147, 0.01729782159336392, 0.025417762621585356, 0.03156066008839251, 0.028656761548796015, 0.023454437450512896, 0.029436876746949867, 0.017403073873417576, 0.018972487311383412, 0.017701130808762396, 0.020658888292230157, 0.027169066561585683, 0.022601918723166248, 0.021302851255249743, 0.030656990937337755, 0.04510613176052095, 0.02669479491507284, 0.04099817905336955, 0.03278319021845058, 0.03245615978545986, 0.027255451332044194, 0.026573065463122127, 0.03175154312611453, 0.0317139288072338, 0.02922178289088212, 0.024761722385618204, 0.021073948945502314, 0.02644495694447892, 0.022044210704972637, 0.020675809009659415, 0.02962661051909138, 0.02111478383947259, 0.01825628341146085, 0.01917605194509881, 0.015144091169859927, 0.013422221833949456, 0.01125626595495288, 0.015099964253611547, 0.0176496043396275, 0.014745517964748305, 0.011873835087600015, 0.01467765269263581, 0.013386228301176981, 0.011928099090551378, 0.011572292756042364, 0.010851777385384325, 0.006817293027784155, 0.008093158600069811, 0.005144998040675947, 0.005292298798926242, 0.0049814836681413175, 0.005497432205449344, 0.005709956829728575, 0.0050965076043312475, 0.005071922832963789, 0.0049726039056405295, 0.02764087036309381, 0.10317966656309088, 0.01083512635592755, 0.00991632044888116, 0.013119007456256009, 0.017882052269343408, 0.013833898371147433, 0.008835462379863981, 0.08469748394721494, 0.010520016271114518, 0.009755990921016436, 0.021323945255707643, 0.026578997822222165, 0.02052581491150249, 0.01306925898903752, 0.01175296137819166, 0.021160525192496667, 0.020990932663456637, 0.0203731049889648, 0.022323217601501143, 0.022744423154106658, 0.014653421125402317, 0.013525431144987408, 0.02596872218234301, 0.02553772137362751, 0.0402570350945573, 0.03015595036030704, 0.03971304927288694, 0.018345400975818948, 0.01848772302043083, 0.017576095979980708, 0.020327928214115408, 0.023418894865925816, 0.024099708272865096, 0.020724138000078086, 0.023244693603618284, 0.018472251219918637, 0.021708200248090444, 0.022059177694156325, 0.02634793083890227, 0.058204002176424546, 0.018827001176662566, 0.016817508344998525, 0.012624335292353414, 0.01780156136773828, 0.020064274035912873, 0.02174758390083288, 0.024374331319048658, 0.0726612198088306, 0.048981847280879606, 0.03477483990473929, 0.0404187978666581, 0.05779112892338689, 0.05610853160930241, 0.028807937983255618, 0.04121997801846587, 0.052763203458449086, 0.05086649237489801, 0.04903882163867762, 0.0520985629141366, 0.04821456980290745, 0.06229520661178768, 0.055098779958204454, 0.05938297529104523, 0.05828809627973632, 0.07638729545846068, 0.06482762470883273, 0.06375509833766053, 0.06576575339988246, 0.04151753615606181, 0.043928046874328754, 0.04104493742725453, 0.05714186096258781, 0.040019528175411524, 0.04025468663653168, 0.0383538771575321, 0.029912668746672904, 0.049061241388564845, 0.03492877019991881, 0.029552815955044446, 0.027447041368179485, 0.03451093849321291, 0.0732114427860569, 0.06690610765692669, 0.07773033631109602, 0.07762007440889669, 0.04975651151796121, 0.047388657916470435, 0.03270748655387275, 0.02735117378755347, 0.021428981126872713, 0.026752873368019067, 0.05285569363957252, 0.026103310888449596, 0.03820777531789649, 0.029094772036973006, 0.030996262226870294, 0.032846163719584896, 0.04030531643162708, 0.07713733260769015, 0.05231084862910416, 0.05401589889558894, 0.04577481393136401, 0.04697477270951659, 0.05390351355375288, 0.06652477757105334, 0.06316571800026786, 0.07360767314517254, 0.07111621851729336, 0.12839781061364156, 0.1316246614929031, 0.07156529784922484, 0.0665363222913083, 0.08250432060230235, 0.07610737143552745, 0.06630111450456504, 0.07254055549839689, 0.05512714344024018, 0.06271580668811362, 0.060458111814331986, 0.07150468244224545, 0.056468806269362196, 0.0553491613385968, 0.0874715393440492, 0.05741338714121744, 0.05515291592990787, 0.04503288513556665, 0.044474633746976175, 0.04247971523664566, 0.03898506985277087, 0.0461131424232239, 0.04110133624387295, 0.16248625342960055, 0.017595769884935244, 0.020336832262655737, 0.027231909305363792, 0.02944945853002631, 0.02000760404915852, 0.018072913221052558, 0.02606480870075696, 0.037958483954385364, 0.027086465670428505, 0.05392597733131174, 0.018470609772037315, 0.029569823067577578, 0.01023730355623877, 0.010474050435129199, 0.020629916053253164, 0.01506987569632097, 0.016106130233319044, 0.008433302894786466, 0.014730645485364244, 0.006611628599135046, 0.009453052480408721, 0.01218185629079621, 0.019311265607038226, 0.019136570463161508, 0.017567148237418215, 0.011172254117177857, 0.01224197807408467, 0.01357781529816418, 0.012576954601870277, 0.013961871769346514, 0.015548945672040886, 0.012370865345720707, 0.012121632405551579, 0.014286679945238141, 0.012960081742390111, 0.011884888042538119, 0.011613088466841623, 0.011262926996367231, 0.012296140984687952, 0.012665985901060965, 0.01435849876145345, 0.011822414594680112, 0.011683503302857482, 0.011667597890690547, 0.010284147120540923, 0.015374456078474022, 0.014359174295767909, 0.012546063559781651, 0.013363614956824248, 0.011381008145168698, 0.013439784697793217, 0.012680591289776992, 0.013175386016336584, 0.012671661318981883, 0.015892878893878783, 0.018691245224727034, 0.01839943304080249, 0.019743487066662353, 0.016046431573627853, 0.040361045405268504, 0.03511400975499439, 0.029210388719702877, 0.03750515852327825, 0.025693408601241592, 0.0212381156178636, 0.022863925145598343, 0.033668377574495104, 0.04117980767057802, 0.03251699660375687, 0.031682968512946975, 0.019471160706850996, 0.03595400641805726, 0.046010752073225, 0.05590042984519225, 0.15345165143436115, 0.06696968091287872, 0.042441984677074183, 0.09032355643637027, 0.08856819495869794, 0.06287512649403106, 0.0483681697126999, 0.05162432598965099, 0.058449879361271266, 0.0770028639842745, 0.062069226130642, 0.05242718817780253, 0.04550578886768619, 0.020329325379468254, 0.02090677671605324, 0.022055220336019515, 0.028216251958800606, 0.019768982312206456, 0.02107784187983984, 0.026387841654479677, 0.02855570899337014, 0.05154625333540479, 0.031320526919948466, 0.02638334849106855, 0.026555096949723574, 0.02863073557770662, 0.030481365906235923, 0.03212334132255493, 0.03240655281091663, 0.03377112929476956, 0.034977282136098596, 0.037753057378205813, 0.03858349480130664, 0.036966457796171574, 0.03472375102414847, 0.03498307894184367, 0.033018537189793695, 0.03261243553024756, 0.030305920798982165, 0.02888669476661888, 0.0275205914766585, 0.028315097477705158, 0.026796200167074156, 0.02576245733888444, 0.026589077145581518, 0.02734304224715327]
  std_diffx = [0.1313722564192988, 0.1369218777584413, 0.1218997632870283, 0.14104932861998826, 0.16055334180740258, 0.17033129739629613, 0.1591849266311516, 0.15921465601303217, 0.1703223275665374, 0.15287712238539694, 0.1550815732699079, 0.14959713544035683, 0.13509466545169294, 0.11330242456344827, 0.12936166228011028, 0.1440322497345232, 0.16428489367608873, 0.18046753417580086, 0.1755136160432313, 0.1551058716515835, 0.15481308978029787, 0.15244366004179272, 0.16067961555908744, 0.16051442667454957, 0.15098594653664615, 0.1379529605516465, 0.1523291466521968, 0.12498288460679477, 0.14503361096924192, 0.12329024818364437, 0.105038782047405, 0.11182836774794006, 0.10474389215012207, 0.06084145754851387, 0.20228179811572458, 0.4795600260027187, 0.0924660793515085, 0.06876389431584443, 0.08591934173725675, 0.07038289008308446, 0.11250006025299897, 0.04845001255141835, 0.06300992883106672, 0.11223792594230315, 0.08609673989537665, 0.15497262407996282, 0.04744927107476604, 0.05718577346005366, 0.11608394714395502, 0.031389147573343545, 0.012523740211201697, 0.025662612158238993, 0.0202306667875588, 0.020268661519425836, 0.06328316424340699, 0.03154884476502039, 0.033861153074045885, 0.033171173189147536, 0.03186111494116176, 0.03292863937193988, 0.04466290155232745, 0.053051725564453096, 0.016678722794996083, 0.05252151192720294, 0.25981364459909123, 0.08630576435867271, 0.04398559594297317, 0.08015704014488421, 0.03975991191817871, 0.027770907728298943, 0.02773289628212088, 0.028065603582375965, 0.02698200648829587, 0.027807551181387984, 0.02772726617359287, 0.026620735649685815, 0.025481476272233426, 0.028012638571692405, 0.026358049373311945, 0.02618983152142333, 0.02281146687529081, 0.020825166707487283, 0.020451824590287663, 0.019926855714089094, 0.017350036059583306, 0.015815870592262558, 0.01598313169172877, 0.015590138900631791, 0.01641342799663984, 0.015591701165784773, 0.013952382657301624, 0.019716162307110225, 0.01497859790096328, 0.014682316558631297, 0.015360501806232328, 0.016302408245853876, 0.016817245503534382, 0.02019000115263552, 0.019579152147623288, 0.01984841464133181, 0.03855919752504424, 0.0163612928511781, 0.018404098222827035, 0.015855783497833748, 0.018791749519368746, 0.018027987855776723, 0.014913753833187587, 0.020915368978481803, 0.02150715218568144, 0.02047360617181268, 0.02088818720639891, 0.025936134988468574, 0.024887423602671897, 0.023150747036880994, 0.025594640253279063, 0.11640488117141702, 0.02301014527397885, 0.024356549220859346, 0.024175658851795474, 0.025194660145030995, 0.08001328146280985, 0.08584496878906779, 0.02985134837991229, 0.026478358171084803, 0.02483701731727526, 0.0858945560279495, 0.014677513962590465, 0.01230248813608093, 0.015095991435309711, 0.015943573178971722, 0.015810859005421073, 0.016462010755508582, 0.01437195960698434, 0.02144714088602238, 0.015073450962166713, 0.016080195219561926, 0.02694728300914301, 0.02171118920041904, 0.021807547651229955, 0.02550997497489851, 0.02099058863759154, 0.021630927522836133, 0.10494238383949095, 0.05277965783364606, 0.035262507139804986, 0.07722025073964309, 0.08711256106703634, 0.1649811014867825, 0.14380357531147817, 0.06389656506623549, 0.08097217719960771, 0.10977611916656212, 0.10990327695036915, 0.07416480023379306, 0.06477989587309839, 0.053380466221838736, 0.053175136952219884, 0.05431806860682666, 0.05969873979492921, 0.06927845424792697, 0.059862259071828974, 0.07225341414739901, 0.07646633063203628, 0.08681410357899642, 0.06440844471953215, 0.061448387845739985, 0.06711145586728812, 0.062179432295279044, 0.0638912300892139, 0.07244815007239881, 0.07332806040919318, 0.05430722310898472, 0.0557970820415573, 0.06964083728531509, 0.032527004881883405, 0.06007391028450617, 0.06601007776947455, 0.09873341849437965, 0.10575257504342939, 0.08450441555531504, 0.07021244111474544, 0.07591661781692266, 0.06279848942295727, 0.057623675706491066, 0.05958390314020923, 0.062333279646500546, 0.06277848482475051, 0.06491252236370904, 0.05896176274059261, 0.06338137432172535, 0.06388218518885602, 0.057824141407890654, 0.0595162565496676, 0.0540509097692303, 0.056446037034606364, 0.05191929160510248, 0.0748934664108333, 0.0816122560617832, 0.0799909983836611, 0.0803999557714407, 0.04972309762819724, 0.04590610477996239, 0.04670931918786799, 0.045300996011068115, 0.04266981802220952, 0.054364644983283625, 0.051716422250284035, 0.04142797238920008, 0.039101890305851525, 0.037775553863852424, 0.03753003151776037, 0.03625592087076482, 0.034608437169158866, 0.03364347240326204, 0.033579105885694356, 0.031914346533882906, 0.03156736990463079, 0.03125752992927064, 0.03022355466717143, 0.029357859515724477, 0.029477897379647042, 0.029767176829101974, 0.028550883024104568, 0.01152531130765381, 0.011694446298690809, 0.012094570644604561, 0.01169256689247426, 0.011022728287257035, 0.011582823302230085, 0.01133798808323584, 0.010881972957506352, 0.02362612907870907, 0.02250508137470681, 0.020290603572871827, 0.0197950260763659, 0.04633661549095997, 0.05522079949508257, 0.04579048209216478, 0.012803960065267928, 0.012742571861651168, 0.021140778780455005, 0.020630560424969524, 0.05384064643416814, 0.06241784256929545, 0.05528331497087472, 0.030758760936033153, 0.030028455909961498, 0.04387294424459081, 0.052929227868001125, 0.06397081740062625, 0.057356269164498715, 0.046948473543060386, 0.034963147354990426, 0.03758973336617662, 0.06942761882286491, 0.07460393578375385, 0.06514682425192142, 0.07226754655458967, 0.05879555558734132, 0.034614910338384326, 0.04106576512082, 0.040658974530595186, 0.0429007237199426, 0.042161306046007076, 0.04109004864742064, 0.04123105011454673, 0.0410811695991903, 0.04012080388639787, 0.0405062225152065, 0.04002606741831761, 0.03706578946869779, 0.1288989166549164, 0.0373097033130881, 0.0361196453818093, 0.034645778442151914, 0.035986856374935504, 0.03538939406746264, 0.03501728053480305, 0.1421867187875678, 0.13201899292773023, 0.03818024597592603, 0.036100065918634995, 0.03719243519925669, 0.031796062901483, 0.0767090692386447, 0.026880223525917814, 0.04068255958678462, 0.12312395178315354, 0.04483562821291177, 0.08253904855172783, 0.0457407884697807, 0.0436401750169184, 0.05652252167907715, 0.05304432357149273, 0.05793661452734573, 0.06133052306465054, 0.17080447492686418, 0.057773468574606156, 0.07154723382370129, 0.069258323237237, 0.04423538166623474, 0.03483781631226951, 0.04659620801670193, 0.04706483099775661, 0.0410643827719499, 0.033994704235372, 0.03402985601510478, 0.04173816972670067, 0.06017464411872224, 0.051830646575699184, 0.032328321331895435, 0.03372156828160129, 0.04076186294759124, 0.13153848490463754, 0.12585921828119437, 0.13744885406922386, 0.13796954015795693, 0.047842152691268465, 0.056061405455101035, 0.046386641656050394, 0.031475638849460516, 0.08880556649868192, 0.13222525550899766, 0.2078973611666329, 0.1308267158852518, 0.1364406607569682, 0.1454122856639755, 0.16194640156465187, 0.1669858947709188, 0.14873743638123008, 0.20105768436106372, 0.13214769257499298, 0.13233869097310808, 0.15313896964315926, 0.11689968962312658, 0.15382304835611743, 0.14437380807979444, 0.06911418036266728, 0.09136613835099161, 0.06626777125986565, 0.11114028223923297, 0.08546048730679835, 0.10315860770716648, 0.0704313923614167, 0.11927870558254135, 0.09492476055152384, 0.07751378303580747, 0.12416347208864034, 0.12054184561806938, 0.1671793345935418, 0.10793618843594405, 0.15894551592484626, 0.10448573726072234, 0.16098852813430595, 0.18054082182907244, 0.17784693029323115, 0.24969917976548736, 0.10475513627629925, 0.11164834900880316, 0.20481977904746487, 0.09467037379149523, 0.1233340083392832, 0.14481668047786514, 0.13327065406082558, 0.07721346816433228, 0.08016287598641415, 0.03332281185003729, 0.02351147778999016, 0.039728970231054095, 0.036634919122812644, 0.03392052365679472, 0.027973631687375714, 0.02958997883434145, 0.11416825438523312, 0.028782823540815974, 0.012522817683165162, 0.023347030195945428, 0.025543902827785205, 0.034734479680061416, 0.031885562441290795, 0.01668869700427501, 0.019154039991540726, 0.020874056151552505, 0.017641402080039854, 0.021169780213061092, 0.02061716846472055, 0.02647793987713792, 0.026348763586546347, 0.04135330601033145, 0.03016231471319019, 0.024675592319929066, 0.02374087841866422, 0.024107037376791818, 0.027899167933025094, 0.028543466326717268, 0.024013700624252624, 0.02407011037248682, 0.023711269742126527, 0.023348130763115094, 0.023290562893106687, 0.024050330157063884, 0.023918915197886262, 0.02403326687601888, 0.02357044266937129, 0.02415820982100022, 0.02305331133705461, 0.023146840528921624, 0.02351472387142671, 0.023052216744279694, 0.025008500725768025, 0.023911037795408964, 0.02205966503735035, 0.020178516002481082, 0.023146771901221223, 0.023895290909842378, 0.023670675082390555, 0.020300059661823928, 0.020767548098352247, 0.023026115041518594, 0.023289156074097365, 0.02315713043119558, 0.02348449676069007, 0.02445224470480859, 0.024842187579857306, 0.01069024282487053, 0.036170983457230764, 0.030285515249984834, 0.0101871713252674, 0.010369926002724433, 0.009239411304808008, 0.04123298116469858, 0.012135120206996962, 0.011634079648042828, 0.0094545779582079, 0.012381297693589597, 0.017635542903277773, 0.11925141314948341, 0.049761107680279644, 0.29420994223097224, 0.14052466073033884, 0.11954347175025049, 0.09994175517630395, 0.09054757684093077, 0.04965108825208485, 0.03143089171277937, 0.04892559824651945, 0.13316134955372888, 0.04976625677212105, 0.11869664397495512, 0.03863381612776653, 0.035540383871700554, 0.02920002411473075, 0.027751237180978994, 0.03001831236177698, 0.02851745867452658, 0.026267081444661017, 0.02774457375130234, 0.03191736252843845, 0.03128640847274431, 0.12907831131786346, 0.09442994992088709, 0.031051703708294416, 0.023384538025008176, 0.023128650951260182, 0.022483776807286228, 0.02343288642298681, 0.02380086009559235, 0.02526993235465801, 0.01734638445887123, 0.016755201455147768, 0.014591798697784316, 0.014210610479578775, 0.014627543210160025, 0.013341282377774289, 0.01202705414110689, 0.011261643267068197, 0.010762402253056878, 0.010043528405220582, 0.008799980911480746, 0.008230169461428126, 0.008490256491053989, 0.007869886998420414, 0.007688679767336991, 0.007589237414566975]

  print("==============================================================================")
  print("The secound exercise(squa)")
  print("==============================================================================")
else:
  pindex=2
  mean_diff =0.1478110747576858
  std_diff = 0.09662747886527086
  mean_diffx = [-0.013074799563790094, -0.014916022945712934, 0.014722216073126324, 0.002730461886425343, 0.13140138380611865, 0.1421082083681455, 0.12766740828960707, 0.10079922681232864, 0.09401238029748925, 0.0985834049907024, 0.09558687749804579, -0.30704367881120026, -0.002120547401755349, 0.007637465742610361, -0.01645267494233141, 0.04446999772232525, 0.10619797651635192, 0.13922103465440988, 0.057307536062424634, 0.007682696148928275, 0.04114875967674722, 0.16949354493043967, 0.14694484676940955, 0.1256771455911363, 0.1962078042989108, 0.09827175965869958, 0.12611716693091973, 0.14833355774321455, 0.14282260695433374, 0.13261327515513022, 0.139663223376174, 0.14692307980666708, 0.17271757243276945, 0.16157188478364515, -0.014847771891343675, 0.17714204545507553, -0.015531764243450657, 0.12603725840375074, 0.13083162511148733, -0.040151227933425, -0.025196741524292597, 0.12043885520496461, -0.025447309124508086, -0.022925092638095484, -0.035000889343002335, 0.16527084916916748, 0.11242061761963207, 0.1321051893790296, 0.13969979476331415, -0.0034981143969108498, 0.006544384220939729, 0.0491290785875907, -0.028004024891865435, 0.19261965825605232, 0.1424631614269838, 0.1399726625299763, -0.016536344232142522, 0.0022187810975403464, 0.0015801832312048755, 0.0044820442491759335, 0.010724653529282197, 0.0031848951045399315, -0.007722319374299187, 0.0076916728714000685, -0.03162137263005859, -0.04131045670183938, -0.02676034306772435, -0.021851794131176182, -0.04519030241051882, 0.15884532685673444, 0.1467567071628641, 0.1510776902795545, 0.16945975235292882, 0.16037634185713961, 0.09624725483217751, 0.09702227165744284, 0.12475207891063621, -0.03970289447636879, -0.04195277146950345, -0.02308199015122547, -0.016128638050920047, 0.15002237746191138, 0.15085524839626452, 0.148131253541431, 0.1601507984623084, -0.036871378312361, 0.15693682557357483, -0.04509699256947356, 0.16606634340290827, 0.3114080267645391, 0.31725078426134296, 0.1414872911650438, 0.18350726891370572, 0.1765776008808395, 0.17497159174658222, 0.1207076421767219, 0.10713072755692041, -0.030507717646928403, -0.025412911304530095, 0.1184726488180544, 0.11864792117241336, 0.1429049258414884, 0.004632288033610417, -0.002532995258951559, 0.0014297720559125926, 0.003215100894863232, 0.020101617509988987, 0.012557525913274383, 0.01761727994192106, 0.13883803446790174, 0.08894468041393187, 0.10103364240697395, 0.0920918923341495, 0.09482196935075195, 0.13455466160579005, 0.14097054330774003, 0.1196191133626841, 0.11898255749111121, 0.13876380203468486, 0.1367385004001339, 0.10911533224041602, 0.10271857724959677, 0.11493604130062873, 0.1121358177629736, 0.11772742190139118, 0.13429623813859556, 0.1478637425419899, 0.1484411959006141, 0.1423435002475436, 0.14042742012784096, -0.0363323930660717, 0.12413780048501964, -0.010946699464064949, 0.07412199137894725, 0.09900241694576106, 0.12116845226037569, 0.13035957370926346, 0.007225217474822292, 0.014560872683710148, 0.027755770045379782, 0.014994617385530389, 0.00924801437851423, 0.011932074693880152, 0.00507433679581361, 0.002320721561663687, -0.0008011301709159269, 0.0036883888744323897, 0.014324069995706101, -0.023159064683156077, 0.007889760733298438, -0.0038690846535388117, -0.01944147114442719, -0.017495515409293907, 0.002341853298998468, -0.009211159412538378, 0.017094618083211194, 0.015909819383490925, 0.010135740827506328, 0.007671693147325467, 0.008445506360517784, 0.003177409286689181, 0.001125713620167545, 0.016974190527220646, -0.06036651566607085, -0.04613380727932376, 0.11346255000270589, -0.018241221646837683, 0.11388125927240804, 0.02733309290619763, 0.01544849255965274, 0.1138792925902075, 0.13086188319950717, -0.028947529625021517, 0.14386985252485815, 0.13645700022858626, 0.12840611437582544, -0.052755029620757996, -0.0022163889942130367, -0.00501180362771561, -0.00680001565264236, 0.12453693757910042, 0.006221558719426083, -0.027794423271271348, 0.011153256902249024, 0.022432106831734567, 0.01641833332706892, -0.015749006957553296, 0.008637131154712348, 0.006288925585077505, 0.11923603248611397, 0.009197466813781568, 0.12011595656613995, 0.1321612987406894, 0.10851591595751729, 0.13611942885064818, 0.16109445238882145, -0.04120729233497504, 0.011121225885511923, -0.013268477967203946, -0.028166517126233633, -0.010187781460334625, -0.01976537926082977, -0.04210973003079323, 0.12018095290602819, 0.11690349231745997, 0.11200316905554858, 0.11798581922059632, 0.04074682797717637, -0.021963035862965025, 0.13031818931418837, -0.05558679060574468, 0.1230809442229794, 0.12056899857135128, -0.04838318455124337, -0.051161366817029104, -0.03744348584824218, -0.05944593987646634, -0.023258236734901506, 0.0568957240730206, -0.017401845961383125, -0.04325018592906385, -0.03700916860263063, -0.044314599278555244, -0.0263494453432935, -0.019004365750340115, -0.025846802574612873, -0.01605918770021175, -0.015025241148162855, -0.015771303567029705, -0.007072992819045771, -0.00880860846164675, -0.002849263929204614, -0.010500593252315758, -0.002381565379401859, -0.006189818205437223, -0.001526583578599881, -0.0032417883148210098, 0.003932001071209858, 0.0015895591307856346, 0.0052629289951914736, 0.0027413414799801784, 0.00420768783512942, 0.0025011778697787094, 0.004354311351124387, 0.0035279440086431564, -0.005239342808943595, -0.0004981168693230934, -0.0007847528925174579, -0.002596286332856769, -0.003051495483546043, -0.004626794254704637, -0.016248887542982374, -0.007957098268065519, -0.0029463675534643057, -0.0005308804006834079, -0.0007694698019273501, -0.0018661748175322792, -0.049381278842168044, 0.0002943655625436484, -0.04558030150481621, -0.030656014915450638, -0.018855397608064275, -0.00180314638740122, -0.006772000918685911, -0.003376989611002924, -0.011315127397855496, -0.003185955016701405, -0.0019148997034415082, -0.03364676889475639, -0.001756721789531691, 0.00982081247290464, -0.0018838361882351734, 0.015200446064216402, -0.029149743872534346, -0.029568078061368804, -0.012336376783191468, 0.10209485054926556, 0.10651294226346977, 0.11166244308345029, -0.04955124178095129, 0.13259370427884137, 0.12934322130249146, 0.13170075119592195, -0.04988919571049455, 0.09867301378268963, 0.11077858723080858, -0.17444880420237785, 0.1584974147966319, 0.1557987091792827, -0.03706999485453675, 0.014485312261958528, 0.11259021187663165, 0.12933677421591003, 0.14397088657807092, -0.01206265494432308, 0.12152350013524986, 0.1564948311837471, 0.2617580493764613, 0.12695036192872275, 0.13381042951632252, 0.1427990259672562, 0.12003800385382551, 0.1522297599564547, 0.14767842135915926, 0.5854376592211057, 0.16017915733908633, 0.15423854980423282, 0.08444414735536662, 0.12783089597235733, 0.14321508974779434, 0.12734329241023454, 0.14849712917264093, -0.010787030498277396, -0.011750338274815364, -0.0003733724931856677, -0.030301923502920828, -0.03456143100412167, -0.039398461042326034, -0.035314102656441214, -0.03160943725773772, -0.02911007238458759, -0.04039920479221209, -0.04897631092620333, -0.00782012926675903, -0.04100257004869071, -0.09237817054428218, 0.03297065233016376, -0.031571546839540915, -0.02345260895114202, 0.01368298319858945, 0.014141793946648768]
  mean_diffy = [-0.03895952884377224, -0.03154857819155448, -0.010577267601781806, -0.021845472529322736, 0.09891521526520973, 0.13237236672779526, 0.13022506728752567, 0.13621593690530553, 0.13682928883047166, 0.1320042797879375, 0.13601579501606803, -0.27568710695913967, 0.04311795469827902, 0.04395968763800252, 0.04837049903575037, 0.03188604575078706, 0.14015368051500676, 0.11732909755570073, -0.004027861618794743, 0.004329151631171808, -0.014005501079639396, 0.07047347660894675, 0.046598628699018665, 0.04589716403021114, 0.12963626789164348, 0.09541586538050904, 0.10599821445966257, 0.11943065886056388, 0.1087313872873123, 0.07373241980200118, 0.0984808918989188, 0.10748857575258447, 0.08734202305018418, 0.07734776586423636, -0.02354687305854429, 0.03798442747107968, -0.04752666965924047, 0.04810186513318338, 0.04991507739054784, -0.13509751627028116, -0.049218625590506944, 0.1238468094533518, -0.010693786620608632, -0.0110177267170252, -0.0033384593607977187, 0.09468727372110557, 0.02817825688560985, 0.0348774422137153, 0.03283831206043465, -0.08211349723447281, -0.06047427536386165, -0.06706454463491023, 0.005423308142802542, 0.057378303282842956, 0.07764911084040432, 0.062131711720345324, -0.03311607337077533, -0.01845067685053754, -0.017852884210992733, -0.016872508693301317, -0.014639047021036512, -0.03374540497397103, 0.00021589486755990372, -0.032361347178586124, -0.034849526013519073, -0.018463013044270428, -0.013611202865082115, -0.01594324717564632, -0.0419542292869338, 0.081155566801758, 0.06165368053455266, 0.06190352507450174, 0.04360680100448432, 0.052364701791591736, 0.04796849581052685, 0.08417413714956527, 0.10156965470483711, -0.016731685112803974, -0.03186210074030019, -0.0184570266764702, -0.042975987085498366, 0.08033818328685287, 0.0574398229995215, 0.023709009679107874, 0.0844465299405489, -0.01164485795569147, 0.07236667178286862, -0.013240519741234793, 0.0747160555346893, -0.194102209543287, -0.18187393338840047, 0.07357134896882167, 0.10929641423980456, 0.10415607947189745, 0.09582784488938882, 0.08195418398733442, 0.06892704793038801, 0.05516384068993362, 0.1011209585124983, 0.14168424627326232, 0.08364545480725322, 0.0615462158337915, -0.0374045336620731, -0.041940424282637945, -0.044600143366857414, -0.046239230469622195, -0.09598800859973856, -0.09271291966295403, -0.11920100026826655, 0.014500961239684385, 0.03324381088596693, 0.0789500087453137, 0.11521120820845869, 0.11966835971107283, 0.11248968509575712, 0.11470685524043439, 0.09749801988022236, 0.09732618871015639, 0.12318556539353236, 0.1245327737853839, 0.13264371414824985, 0.11332294953112619, 0.12042188001736352, 0.09650949218320302, 0.07604595400357927, 0.09625787268384514, 0.1134893483723183, 0.10779795002826517, 0.10990051570412049, 0.11616206682639361, 0.028666728957732403, 0.1215114761104739, 0.05539471049297585, 0.09237164391894546, 0.1259791170104073, 0.12844489942971576, 0.13116003620842162, 0.026432399316749268, 0.011914705828178005, -0.009916141266473682, 0.01664647587114805, -0.0032984755499812363, -0.01733335706802686, -0.030383444243317445, -0.048750130896020184, -0.060761219491610924, -0.05369131605439146, -0.10954659480920291, -0.14934808942355207, -0.1273885561011896, -0.06594610036773547, -0.05755640356250993, -0.047736137979932716, -0.001912415883317129, -0.019191932881167745, -0.044681576580517746, -0.10146323912933898, -0.0576380794735061, -0.08414501959433625, -0.05948564084391243, -0.08179427872029969, -0.04671054305486108, -0.10866427086967946, -0.09946478762993684, -0.07535380301748523, 0.010319246342184885, -0.04354068749565771, 0.05760018931118807, -0.09043462953471652, -0.1126447425764156, -0.009361782705564349, -0.026007224938627446, -0.03501257251852971, 0.06022034786863492, 0.07634548332888529, 0.05179967520361394, -0.04290826903003731, -0.029328416079889664, -0.0493417774109055, -0.024748937055287204, 0.05209871945223266, -0.07408787498666161, -0.07900846511607734, -0.08563187533747821, -0.09716438813034635, -0.06571471306120535, -0.04335128890863661, -0.05072129804673882, -0.03176540610183924, 0.04990183380998934, -0.05281145944764098, -0.005619185095693027, -0.026765342168694667, -0.06019022717452167, 0.08438742914019885, 0.08774950146647459, -0.04351974472806532, 0.0006980930096350247, -0.013877203321975413, 0.015355429369991862, 0.01968346307624155, 0.020712196926033002, -0.028675205267419224, 0.02989845405981062, -0.005234703879839721, -0.05139967062438977, -0.04489146960040389, 0.05209148633243274, -0.1656512775348593, -0.09521954538844339, -0.04053254596278258, -0.04116142427064961, -0.05628088889926055, -0.1724345141893123, -0.1913048172380649, -0.1827351028788009, -0.15529032497021034, -0.10011832911312467, 0.021312943030381087, 0.052616302796283175, -0.10289327688427424, -0.14997862275164828, -0.10375051228719176, -0.13400295144495492, -0.15533082440161244, -0.16047913927535642, -0.13458645822314128, -0.12104167931142525, -0.06292544947446208, -0.03427022501484214, -0.013669800393150236, -0.015659869861752596, 0.04308621389326726, 0.02005408658999269, 0.002865220592761054, 0.0020133198903992573, 0.004300359980952218, 0.007719303054254651, 0.0019477582509666485, -0.004465038047496231, -0.004111813208838971, -0.0007226285549203219, 0.0009639464153608907, 0.002549969734523917, 0.00872587440873358, 0.0006441542894802377, 0.005312736146584712, 0.005928212974569892, 0.0030212409920712655, 0.003261613200149619, 0.002375904864938299, 0.0008910095262523618, -0.003126326799254857, -0.0002832398060292071, -0.00629172433120297, -0.005013002623899794, 0.0021251591126321655, -0.053174376377299916, 0.017133419906468172, -0.04864900011575179, -0.020422537684054342, -0.0150625145905581, -0.004797220662159466, -0.01143493012622987, -0.005177957833351736, 0.006098871799286627, 0.004484217733352162, 7.813866803640122e-05, -0.02280960923190765, 0.013994429404635005, -0.005205314403515271, 0.0005902460679275212, 0.018238562234983338, -0.014041801544344076, -0.02799305357643406, -0.01321401750447116, 0.05494418472276084, 0.017748984114572597, 0.03660281627069244, -0.08810427300926758, 0.08019394247482785, 0.07632063657272324, 0.0675630263621545, -0.026516224864245107, 0.10314028325065229, 0.04912508243617287, -0.12217568571729809, 0.14718149916784096, 0.12535867499027012, 0.09214972510844038, 0.0768491003088957, 0.12121831157907156, 0.11357277238190377, 0.10933907203026884, -0.015033442287852573, 0.09069377474723467, 0.10232677196771625, 0.2370435219071091, 0.11271611800387621, 0.12320250366747945, 0.11781142873391659, 0.07932379008195951, 0.1075115021004115, 0.13531995455556625, -0.11172019326958914, 0.12566941127138143, 0.13764539790110442, 0.08033116418279589, 0.1099459184436669, 0.14219561227279498, 0.15280863170129577, 0.12922320337372611, 0.007445311890763268, 0.011767180162200255, 0.01963250927685938, -0.021855996354481117, -0.032584809319921, -0.027910965160771958, -0.030741983363281652, -0.0398778615661559, -0.03030755263218821, -0.06330203278067655, -0.07033233926678573, -0.05617766306523197, -0.08267068549168767, -0.049010334147721865, -0.03794966071097263, -0.05569606922268762, -0.03414372505325927, -0.02223479965489055, -0.020334537901359064]
  std_diffy = [0.05538667278973958, 0.050302425423117095, 0.03763892764673096, 0.03266887400661218, 0.1289324404803277, 0.1517098883627641, 0.1496354262687871, 0.1605253070755041, 0.16195735591342952, 0.152792522978155, 0.15884693317770351, 0.3390678682834351, 0.12436163508304951, 0.10912824884590965, 0.10889472936846396, 0.11152697533302085, 0.18028758920471585, 0.13750944215375915, 0.014667953088491734, 0.03131397464128973, 0.04449480978752377, 0.11590620853671131, 0.143115021477024, 0.120104525645759, 0.18032282056418616, 0.1405720139657217, 0.1455720957755062, 0.1474606461136989, 0.12479018700814329, 0.08752476583234467, 0.12170058342909233, 0.12669091960188333, 0.11341643720373867, 0.12042120596739692, 0.03477766389938958, 0.1300747028807858, 0.0554240746503046, 0.12254154886103137, 0.1070347058515185, 0.15463807919612085, 0.06433408216970715, 0.14696909317451515, 0.026545197412748096, 0.036158280101378525, 0.028203549627214605, 0.14238369582244367, 0.06684510802932697, 0.1562440731889063, 0.12323836378422875, 0.10967933670179847, 0.0914233716125216, 0.08336519393931568, 0.023492168560523035, 0.10433461748826232, 0.09252708949589915, 0.11656773425249843, 0.04320384143403177, 0.06228683022815112, 0.06447449935876003, 0.0658783560187934, 0.054942890118738597, 0.0680697033059666, 0.051334411202851725, 0.06844936696391724, 0.04489720168903357, 0.032638671547038166, 0.025414310680899322, 0.023013308334422427, 0.05836251056879412, 0.11995514587540188, 0.1323758291271636, 0.12806355476261308, 0.14303222854189548, 0.15089636839159207, 0.10279052773638489, 0.14388564161059714, 0.13219102281470155, 0.02938597482614944, 0.04697888984407603, 0.020160935305769664, 0.04631088037238788, 0.13347031429866596, 0.1226966792794254, 0.12533365768412802, 0.14960327614570906, 0.036341266234415484, 0.17082538527592117, 0.04563574558930669, 0.18077440597379726, 0.2239999106049717, 0.22029690112083491, 0.11060839631285456, 0.13603304531883129, 0.12868308921112115, 0.12687972541808198, 0.11977994917738215, 0.12449944539681819, 0.0790088962874409, 0.15040619205144937, 0.16939118343223594, 0.15507567136353517, 0.12499723213277723, 0.08516777915574843, 0.15494596284029824, 0.22417222737096545, 0.16829302876109514, 0.2799865098469392, 0.2979336598397984, 0.3025086713783429, 0.22585411684617376, 0.20054483854016994, 0.20537501289380308, 0.17970368523940633, 0.16941694507855012, 0.1481252264757329, 0.14061808632979833, 0.11140735242256473, 0.1184613545124517, 0.1442008998610206, 0.14349623665950484, 0.15820837672395338, 0.1577812969541557, 0.1526743377266914, 0.1389364285470096, 0.11189982515277239, 0.11689726553333198, 0.13452338255033056, 0.12668636687132534, 0.12970948713904792, 0.13734988488550687, 0.0582471315932504, 0.14144175545980656, 0.1387760595082626, 0.12435082832950141, 0.16971578498121162, 0.15662799578279368, 0.15473629398740626, 0.030530362848658482, 0.019278260758360068, 0.026333974238310708, 0.03466270423712523, 0.02089988407803354, 0.03275168207331853, 0.08856503222280093, 0.12461643283330229, 0.12919838359943578, 0.13624196561820368, 0.24250876130126883, 0.284852608495835, 0.2639521226858659, 0.19843621543795756, 0.22985267071905482, 0.24571148079593932, 0.3050557988069122, 0.18135598413449716, 0.22935802426764973, 0.27060040788562695, 0.2623984245482947, 0.2320922268271904, 0.225712487810548, 0.2122541053182774, 0.14281639761840972, 0.2358375688570735, 0.1849940756497007, 0.13592984387634763, 0.16737263826378798, 0.10978470517037633, 0.13648478444184833, 0.1933106210055972, 0.21365118006929712, 0.19787614432588424, 0.25336120508153875, 0.16903682818421398, 0.19759680931709844, 0.23872415588794801, 0.20725116771424068, 0.24807546824814208, 0.09926510527284481, 0.23702934568647802, 0.26800544425277106, 0.20388148893312968, 0.22288584577579748, 0.21138763967695093, 0.20056682233643625, 0.19386771788670118, 0.1428054491010655, 0.11191875624873265, 0.11663079901439376, 0.08401354508111375, 0.13185330089986647, 0.11966516808476621, 0.20138335375519648, 0.23065573373461348, 0.27812361116210754, 0.2061807404672847, 0.2269533939441348, 0.22293473605653324, 0.10175161281626947, 0.1216983714254444, 0.28080801111816384, 0.28841074905903263, 0.30073808998148543, 0.28641728871866456, 0.2804178640845057, 0.29447482048022905, 0.3480909729435528, 0.2994356938470553, 0.10746859879152292, 0.4012656954572016, 0.4188563851060894, 0.13411938485074723, 0.3013826021178844, 0.33043509324798603, 0.4220720263763529, 0.45919376243690646, 0.4974103229060322, 0.5234285866756487, 0.45702677172000916, 0.40683240632729323, 0.13542560761558273, 0.4241336371460476, 0.42850536950450085, 0.4575251140190178, 0.42858062562260413, 0.3834796201705186, 0.3200809538752888, 0.235482217076989, 0.20892833923935036, 0.11916370445827645, 0.07576484757486221, 0.058191644717888256, 0.023480858158902766, 0.10270819113137995, 0.03851938016575055, 0.006772503329517101, 0.004035594689956201, 0.005361251170440556, 0.011113785338425348, 0.004002382293180914, 0.01819649631347324, 0.018175657222650948, 0.009275400990442474, 0.011726895743973521, 0.0060963006423317995, 0.011055422138074634, 0.006107563633765774, 0.00469285933450324, 0.007608644040335859, 0.006041478856830576, 0.004516284838714685, 0.009805343592580736, 0.010918858900731205, 0.010373851655018614, 0.013291811024874787, 0.013733756510966056, 0.013139724381082093, 0.004894240395990953, 0.05807517856534253, 0.021896044769863517, 0.05941361992673993, 0.031731407359191846, 0.02272433823109772, 0.018013171833277462, 0.02095944006475458, 0.016782343480580865, 0.019020978567156625, 0.009605031596287124, 0.007300832322938697, 0.029309073034600047, 0.018186703038201864, 0.006814354563914599, 0.005620112913312959, 0.02515508563417357, 0.03276847276255417, 0.03616200119025216, 0.020228975380606196, 0.13316627565963593, 0.1805992463142158, 0.1340716881906285, 0.19068416328744164, 0.1710645336216156, 0.17671089808737117, 0.19702053763056188, 0.19661568328115353, 0.21864112298638402, 0.08092076760012015, 0.22462857765018074, 0.21821137533021798, 0.18750758512143778, 0.15685518096773335, 0.10961059725085484, 0.1581718225608211, 0.14431638440913144, 0.1357105295924676, 0.028007774729021877, 0.12703915948774944, 0.13652661031941826, 0.3405228286297406, 0.1638242888391944, 0.16847695379508446, 0.15746899995575875, 0.11176591645324774, 0.15651109557544465, 0.18079306989807356, 0.12022062738529148, 0.1693102821201341, 0.17668848390652034, 0.11281489283723276, 0.1602602853299896, 0.1883670329058655, 0.19652140965154183, 0.16582887903904006, 0.027634648739276516, 0.03688470521611876, 0.026054741837061048, 0.03114575734499222, 0.03855400254119492, 0.03565892796235985, 0.04406782913130469, 0.060420057600285765, 0.04491664778158625, 0.07592366836915404, 0.09341385955244062, 0.08852708895468947, 0.12041547180002371, 0.07875175265191924, 0.1266812400774084, 0.1073840037947796, 0.09268312379081309, 0.07894999995064288, 0.06092628768292096]
  std_diffx = [0.018883836381095295, 0.020837247909364025, 0.02960136047197283, 0.015283568653075463, 0.06361661274956129, 0.06603694166052476, 0.0648338855558013, 0.06903765741972044, 0.07971796454267623, 0.07937809308064177, 0.07821264520710589, 0.17380290383346186, 0.017847785736411187, 0.010400858857090349, 0.11308551937600893, 0.1300893154780423, 0.13452527519354537, 0.0655743424296212, 0.15674182991548266, 0.02782792072828304, 0.12969078613294394, 0.12324204912560366, 0.06678384284987428, 0.07938463876623085, 0.1406612513272692, 0.09547649262821083, 0.08346919462411573, 0.06369830232955591, 0.06420046914651158, 0.08181738046387305, 0.07553909499175537, 0.07004191603896943, 0.12550239418056586, 0.13479452439300219, 0.04088545687786137, 0.1677974568388591, 0.022150533135503383, 0.07148990467825177, 0.07280036434454107, 0.06418956836927556, 0.021912060214850947, 0.0628854067318626, 0.023694690677795664, 0.023574047579093513, 0.05023020417414801, 0.14793557708737237, 0.07468186476207628, 0.0757784884211786, 0.0789380477810472, 0.019250754944488656, 0.018527661825033587, 0.15325615027237782, 0.07026831600808187, 0.1115264285087428, 0.06397466745709868, 0.06499433731929197, 0.06367222259883315, 0.02017368202626373, 0.015629876879717966, 0.01572490421180985, 0.030464863773007275, 0.015514415223029342, 0.010209803751624913, 0.014005545706796026, 0.046076044200721214, 0.05983609107542857, 0.06115925463588302, 0.05770439561682323, 0.05811081382955427, 0.11607002627976706, 0.0696876787189921, 0.0701358109488243, 0.10888672982289137, 0.11582308478761542, 0.07792914879246829, 0.0909601528202348, 0.12282841256842027, 0.05711155755717538, 0.05517231910360249, 0.028378379730708295, 0.020277059438093322, 0.0694165746802079, 0.080998882276573, 0.08099428937501903, 0.14947277562636965, 0.07523298477965419, 0.16546763573859902, 0.06275415671831737, 0.13755455900564853, 0.3053217846308407, 0.2965020987566572, 0.10758663346320434, 0.12158682918038352, 0.09672649131445318, 0.13398329298628545, 0.08393712532320234, 0.07236291922518712, 0.1509574680730357, 0.1135376326534454, 0.07380224010239213, 0.08000312784696624, 0.044459989836737795, 0.07682252335096586, 0.01607057732749146, 0.022911471062037385, 0.016726666086924916, 0.027877241209142364, 0.04102823639240021, 0.03390558578909429, 0.08446822357131768, 0.08478049449833888, 0.0973718263277057, 0.0973393779251381, 0.09508098726069912, 0.07825122045471153, 0.06633378189569926, 0.054511097419785826, 0.056744735390692094, 0.06312866008912973, 0.061515114361419716, 0.0690627747861522, 0.07704548956318263, 0.07588018209668217, 0.08261066049679996, 0.07992157068168182, 0.07474309588002641, 0.06906654053677755, 0.06897683455900609, 0.06576516986592615, 0.06557573534063649, 0.12973385866919446, 0.0750785746697458, 0.10125771848758139, 0.06975550027845961, 0.09528047268629596, 0.08398515145555617, 0.07550147584920132, 0.022176431601689434, 0.02603710030719844, 0.05895331567433433, 0.025147729802950977, 0.02513820041135411, 0.018488078187928275, 0.016354348059543576, 0.019741915215744057, 0.0334227458604607, 0.030982911657715276, 0.03544429868606176, 0.14020881717359326, 0.038205602243685056, 0.016604533504199572, 0.11010408821871522, 0.11492060687353874, 0.039085539193826346, 0.037094616470331215, 0.04121622609515764, 0.053355258771498125, 0.039345992558227384, 0.03218759042835343, 0.039336806919693004, 0.04568852977217113, 0.013401273035511283, 0.09836115946805592, 0.14582134245921147, 0.1252912135758559, 0.07400571491641444, 0.15758377947845179, 0.08221618654485653, 0.058810191898964505, 0.03355860024429664, 0.07325143348962632, 0.06591845931762687, 0.10767690948234093, 0.06700482413830329, 0.06621174683017561, 0.06187844007560964, 0.1387072753913962, 0.017296154004416003, 0.02940129904031501, 0.026876553438067496, 0.07360068790898769, 0.023672777949603565, 0.13968257771383785, 0.04188744927534354, 0.04611329017480239, 0.04384869708403391, 0.13549945350896758, 0.02023840480119672, 0.018320657074745166, 0.07872952138922046, 0.028244962853648376, 0.0790852758184544, 0.0671056134558731, 0.05875540824103007, 0.06353573136540763, 0.08392164993847995, 0.11295506816222126, 0.015947049012141293, 0.07541043866529555, 0.11434801435337566, 0.02277150200588996, 0.09213128243410638, 0.11504251592130099, 0.06096267564397882, 0.05732452475386566, 0.07100679642320631, 0.07047542417050243, 0.13306195204281454, 0.15121348062784215, 0.05921137470761831, 0.12951669902557367, 0.06546231808352199, 0.06566618174270057, 0.12708082404576437, 0.13011819152309495, 0.11061044072866974, 0.1339493765772245, 0.08013589716413161, 0.09010685627511479, 0.03530474657735387, 0.06508730396244983, 0.0625289916151236, 0.06087692606586683, 0.0840528083819411, 0.07684904501256289, 0.04044389459361608, 0.031877255037864996, 0.029111869494474352, 0.016726310089475345, 0.013081221092864505, 0.017581728227556204, 0.020149991266525155, 0.020932593858233643, 0.00968694853172353, 0.013806898607961129, 0.022161146946717607, 0.009893742366485986, 0.0184291725616926, 0.007329224121069121, 0.03382748565734088, 0.008181508059679632, 0.007150406050332497, 0.022034512759029037, 0.008369503020172989, 0.009039717224043071, 0.019395408787911808, 0.006006381219946693, 0.008543625200862234, 0.00786044988710368, 0.007219418356708895, 0.027022811515940392, 0.02499176297512735, 0.017377232137998518, 0.010310279776693432, 0.010488261212681418, 0.009463998760894153, 0.01505591138218947, 0.03198417602820745, 0.02548009845243503, 0.01912248819324824, 0.019864909527912745, 0.021270517779886852, 0.021884332307277675, 0.023797016723879322, 0.023190326028316856, 0.032064993226654265, 0.021871683423893715, 0.0159263669892464, 0.023734119318642354, 0.0069510756533888675, 0.03317000123985715, 0.007890441539718778, 0.06456277861828257, 0.02314344848071576, 0.021283255322772605, 0.02103544271917376, 0.06035152089709124, 0.10626332360797357, 0.06851584827864636, 0.1347735228062515, 0.06657379399113315, 0.0662894465366372, 0.0644375670223179, 0.1368044231631104, 0.07653767706147184, 0.06472271827592445, 0.15940246052660273, 0.1429641295692552, 0.14604643219371727, 0.17057143109580575, 0.1225386312070647, 0.14716571088748137, 0.11488472693779316, 0.1079222410960411, 0.14926385242824586, 0.13226332898757268, 0.13113913860292362, 0.16251341139866354, 0.12095092417160909, 0.11900176142769789, 0.11724245887914711, 0.11010297935258462, 0.11044483568115358, 0.11009191071219518, 0.27429150883929204, 0.10879145133560121, 0.10870051966605128, 0.07659413505127985, 0.12888644796080762, 0.12739579294921552, 0.13129747433096703, 0.11556364029603948, 0.10714896645365724, 0.10747219078407086, 0.007781022943899431, 0.011903793919335078, 0.01515688680089394, 0.019812078926743436, 0.019333473199253617, 0.026980403362058678, 0.023727735908631153, 0.020905320398500262, 0.025076605392840387, 0.03696200390374977, 0.023241989101958967, 0.09706798981897917, 0.13363348067754352, 0.024829967202166358, 0.024424574044314334, 0.03728814542621629, 0.041005731825995644]

  print("==============================================================================")
  print("The Third exercise(jogging)")
  print("==============================================================================")
print(paths[0])
# set the path of the feed video to the closest match
feedpath= paths[cls.index(min(cls))]
print(feedpath)
# get the index of the closest match
nearst=cls.index(min(cls))

# initialize empty list for storing differences between feed video and exercise video
feed_space=[0]*len(feedpath)

# loop over the path of the closest match
for i in range(len(feedpath)):
  # compute the difference between the feed video frame and the exercise video frame at the current indices
  feed_space[i]=subf(x[feedpath[i][0]],y[nearst][feedpath[i][1]],pats[cls.index(min(cls))]+'/')

print(feed_space)
feed_space = np.array(feed_space)
# Identify outliers by finding values that fall outside of 3 standard deviations
outliers = np.where(np.abs(feed_space - mean_diff) > 3*std_diff)
print(outliers)
outliers=sorted(outliers[0])
print(outliers)
print(len(outliers))
# open the input video
cap = cv.VideoCapture('feed_test.mp4')
# Define the codec and create VideoWriter object
width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv.CAP_PROP_FPS)

fourcc = cv.VideoWriter_fourcc(*'XVID')
out_v = cv.VideoWriter('/content/feed_testf.mp4', fourcc, fps, (width,  height))

i = 0
c = 0
w=0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    if c < len(outliers):
        if i == outliers[c]:
            
            fr_json = out[feedpath[outliers[c]][0]]
            x_cor = []
          
            for w in fr_json:
              if ( fr_json[str(w)] !=[]):
                x_cor.append( fr_json[str(w)][0] )
              else:
                x_cor.append( 0 )

            y_cor = []
            for w in fr_json:
              if ( fr_json[str(w)] !=[]):
                y_cor.append( fr_json[str(w)][1] )
              else:
                y_cor.append( 0 )
           
            x_cor = np.array(x_cor)
            y_cor = np.array(y_cor)
            # Open file a and read its contents into a JSON object
            f = open(pats[pindex] + y[pindex][feedpath[outliers[c]][1]], 'r')
            x3 = json.loads(f.read())
            # Extract the part candidates dictionary from the JSON object
            orign = x3['part_candidates'][0]
            f.close()
            x_fed = []
            y_fed = []
            for w in orign:
              if ( orign[str(w)] !=[]):
                x_fed.append( orign[str(w)][0] )
              else:
                x_fed.append( mean_diffx[feedpath[outliers[c]][1]] )

            
            for w in orign:
              if ( orign[str(w)] !=[]):
                y_fed.append( orign[str(w)][1] )
              else:
                y_fed.append( mean_diffy[feedpath[outliers[c]][1]] )
            
            x_fed = np.array(x_fed)
            y_fed = np.array(y_fed)
            outliers_nodx = np.where(np.abs(x_fed - mean_diffx[feedpath[outliers[c]][1]]) >  10*std_diffx[feedpath[outliers[c]][1]])
            outliers_nodx = outliers_nodx[0]
            #print(mean_diffx[feedpath[outliers[c]][1]])
            #print(std_diffx[feedpath[outliers[c]][1]])
            if outliers_nodx.size > 0:
                outliers_nodx = sorted(outliers_nodx)
                for j in range(len(outliers_nodx)):
                    frame = error(x_cor[outliers_nodx[j]], y_cor[outliers_nodx[j]], frame)

            outliers_nodx=[]
            outliers_nodx = np.where(np.abs(y_fed - mean_diffy[feedpath[outliers[c]][1]]) >  10*std_diffy[feedpath[outliers[c]][1]])
            outliers_nodx = outliers_nodx[0]

            #print(mean_diffy[feedpath[outliers[c]][1]])
            #print(std_diffy[feedpath[outliers[c]][1]])

            if outliers_nodx.size > 0:
                outliers_nodx = sorted(outliers_nodx)
                for j in range(len(outliers_nodx)):
                    frame = error(x_cor[outliers_nodx[j]], y_cor[outliers_nodx[j]], frame)

            #cv.imwrite('feed'+str(i)+'.jpg',frame)
            out_v.write(frame)
            c += 1


        else:
           print(i)
           print(feedpath[outliers[c]][0]) 
           out_v.write(frame)
    else:
       
        out_v.write(frame)
    #cv.imshow('frame', frame)
    i += 1
    if cv.waitKey(1) == ord('q'):
        break
# Release everything if job is finished
cap.release() # release the input video
out_v.release() # release the output video
cv.destroyAllWindows() # destroy all windows created by OpenCV comment this code


Streaming output truncated to the last 5000 lines.
feed_test_000000000309_keypoints.json Jogging1_000000000223_keypoints.json /content/drive/MyDrive/jogging_bary//
feed_test_000000000309_keypoints.json Jogging1_000000000224_keypoints.json /content/drive/MyDrive/jogging_bary//
feed_test_000000000309_keypoints.json Jogging1_000000000225_keypoints.json /content/drive/MyDrive/jogging_bary//
feed_test_000000000309_keypoints.json Jogging1_000000000226_keypoints.json /content/drive/MyDrive/jogging_bary//
feed_test_000000000309_keypoints.json Jogging1_000000000227_keypoints.json /content/drive/MyDrive/jogging_bary//
feed_test_000000000309_keypoints.json Jogging1_000000000228_keypoints.json /content/drive/MyDrive/jogging_bary//
feed_test_000000000309_keypoints.json Jogging1_000000000229_keypoints.json /content/drive/MyDrive/jogging_bary//
feed_test_000000000309_keypoints.json Jogging1_000000000230_keypoints.json /content/drive/MyDrive/jogging_bary//
feed_test_000000000309_keypoints.json Jogging

In [ ]:
#old Normalization

from google.colab import drive
import json
from math import sqrt
import os
drive.mount('/content/drive')
pat="/content/output/"
lof = os.listdir(pat)#['batata.json', 'sese.json']
for i in range(len(lof)):
  if lof[i] ==".ipynb_checkpoints":
    lof.remove(lof[i])
    break
print(lof)
#j=1
for j in range(len(lof)):
    print(j)
    f=open(pat+lof[j],'r+')
    x= json.loads( f.read())

    pts=x['part_candidates'][0]
    orignx=pts["1"][0]
    origny=pts["1"][1]
    
    const =sqrt(((orignx-pts["8"][0])**2)+((origny-pts["8"][1])**2))
    for i in range(0,25):
        if ( pts[str(i)] !=[]):
            pts[str(i)][0]=(pts[ str(i) ][0]-orignx)/const
            pts[str(i)][1]=(pts[ str(i) ][1]-origny)/const
   # print(pts)
    jprint=json.dumps(pts)
    f.seek(0)
    f.write('{"part_candidates":[')
    f.write(jprint)
    f.write("]}")
    f.truncate()
    f.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: ignored

In [ ]:
#fore pary test

# import necessary libraries
from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import numpy as np
import os
import json
from math import sqrt
from google.colab import drive
import cv2 as cv

# mount Google Drive
drive.mount('/content/drive')

# set paths for directories containing exercise data
pats=["/content/drive/MyDrive/jump_bary/","/content/drive/MyDrive/squat_bary/","/content/drive/MyDrive/jogging_bary/"]
pat="/content/output/"


# get list of files in the first directory
x = sorted(os.listdir(pat))

# get lists of files in the other directories
y = []
y.append(sorted(os.listdir(pats[0])))
y.append(sorted(os.listdir(pats[1])))
y.append(sorted(os.listdir(pats[2])))

# remove any files that are not JSON files
for i in range(len(x)):
  if x[i] ==".ipynb_checkpoints":
    x.remove(x[i])
    break
for j in range(len(y)):
  for i in range(len(y[j])):
    if y[j][i] ==".ipynb_checkpoints":
      y[j].remove(y[j][i])
      break

# initialize empty lists
cls=[]
paths=[]
def error(wrong_joint_x, wrong_joint_y, frame):
    # This function draws a red circle at the position of the wrong joint in the given video frame
    radius = 10
    color = (0, 0, 255)  # red in BGR format
    thickness = -1  # fill circle
    cv.circle(frame, (wrong_joint_x, wrong_joint_y), radius, color, thickness)
    return frame


def dp(dist_mat):
    """
    Find minimum-cost path through matrix dist_mat using dynamic programming.

    The cost of a path is defined as the sum of the matrix entries on that
    path. See the following for details of the algorithm:

    - http://en.wikipedia.org/wiki/Dynamic_time_warping
    - https://www.ee.columbia.edu/~dpwe/resources/matlab/dtw/dp.m

    The notation in the first reference was followed, while Dan Ellis's code
    (second reference) was used to check for correctness. Returns a list of
    path indices and the cost matrix.
    """
    # This function implements dynamic programming to find the minimum-cost path through a distance matrix.
    # It returns a list of path indices and the cost matrix.

    N, M = dist_mat.shape

    # Initialize the cost matrix
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1, N + 1):
        cost_mat[i, 0] = np.inf
    for i in range(1, M + 1):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_mat[i, j],  # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = N - 1
    j = M - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))

    # Strip infinity edges from cost_mat before returning
    cost_mat = cost_mat[1:, 1:]
    return (path[::-1], cost_mat)


def subf(a, b, pat2):
    # Open file a and read its contents into a JSON object
    f = open(pat + a, 'r')
    x3 = json.loads(f.read())
    # Extract the part candidates dictionary from the JSON object
    orign = x3['part_candidates'][0]
    f.close()

    # Open file b in directory pat2 and read its contents into a JSON object
    f = open(pat2 + b, 'r')
    x2 = json.loads(f.read())
    # Extract the part candidates dictionary from the JSON object
    new = x2['part_candidates'][0]
    f.close()

    # Initialize variables for distance calculation
    const = 0
    avg = 0

    # Iterate over all body parts (25 in total)
    for i in range(25):
        # Check if both the original and new dictionaries have entries for this body part
        if ((orign[str(i)] != [] and len(orign[str(i)]) >= 2) and (new[str(i)] != [] and len(new[str(i)]) >= 2)):
            # Calculate the Euclidean distance between the two body part locations
            const += sqrt((orign[str(i)][0] - new[str(i)][0]) ** 2 + (orign[str(i)][1] - new[str(i)][1]) ** 2)
            # Increment the counter for valid body parts
            avg = avg + 1

    # Calculate the average distance between corresponding body parts
    const = const / avg
    return const




# loop over each directory containing exercise data
for c in range(len(y)):
  
  # create distance matrix for each file in the current directory
  dist_mat = np.zeros((len(x), len(y[c])))
  for i in range(0,len(x),10):
        for j in range(0,len(y[c]),10):
            print(x[i],y[c][j],pats[c]+'/')
            dist_mat[i, j]=subf(x[i],y[c][j],pats[c]+'/')
  
  # find the shortest path through the distance matrix
  path, cost_mat = dp(dist_mat)
  
  # append the cost and path to their respective lists
  cls.append(cost_mat[len(cost_mat)-1][len(cost_mat[0])-1])
  paths.append(path)

# determine which directory had the closest match and set variables accordingly
if(cls.index(min(cls)) ==0):

  print("==============================================================================")
  print("The first exercise")
  print("==============================================================================")
elif(cls.index(min(cls)) ==1):

  print("==============================================================================")
  print("The secound exercise")
  print("==============================================================================")
else:

  print("==============================================================================")
  print("The Third exercise")
  print("==============================================================================")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
TestingVid1_000000000000_keypoints.json JumpingJacks1_000000000000_keypoints.json /content/drive/MyDrive/jump_bary//
TestingVid1_000000000000_keypoints.json JumpingJacks1_000000000010_keypoints.json /content/drive/MyDrive/jump_bary//
TestingVid1_000000000000_keypoints.json JumpingJacks1_000000000020_keypoints.json /content/drive/MyDrive/jump_bary//
TestingVid1_000000000000_keypoints.json JumpingJacks1_000000000030_keypoints.json /content/drive/MyDrive/jump_bary//
TestingVid1_000000000000_keypoints.json JumpingJacks1_000000000040_keypoints.json /content/drive/MyDrive/jump_bary//
TestingVid1_000000000000_keypoints.json JumpingJacks1_000000000050_keypoints.json /content/drive/MyDrive/jump_bary//
TestingVid1_000000000000_keypoints.json JumpingJacks1_000000000060_keypoints.json /content/drive/MyDrive/jump_bary//
TestingVid1_000000000000_keypoints.json JumpingJacks

In [ ]:
# to copy files and folders form and to drive

from google.colab import drive

import os
drive.mount('/content/drive')
drive.mount('/content/drive', force_remount=True)
!cp -r   "/content/drive/MyDrive/feed_test.mp4" "/content"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/drive


In [ ]:
# classification befor BaryCenter

from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import numpy as np
import os
import json
from math import sqrt
from google.colab import drive

drive.mount('/content/drive')
pat="/content/out/"#todoooooooooooooooooooooo
pat1="/content/drive/MyDrive/Jumping Jacks/"#todooooooooooooooooo
pat2="/content/drive/MyDrive/e2/"#todooooooooooooooooo
pat3="/content/drive/MyDrive/Jogging/"#todooooooooooooooooo
x = sorted(os.listdir(pat))#['ba)tata.json', 'sese.json']
e1=sorted(os.listdir(pat1))
e2=sorted(os.listdir(pat2))
e3=sorted(os.listdir(pat3))

y = []

for i in range(len(e1)):
  y.append(sorted(os.listdir(pat1+e1[i]+'/')))

for i in range(len(e2)):
  y.append(sorted(os.listdir(pat2+e2[i]+'/')))

for i in range(len(e3)):
  y.append(sorted(os.listdir(pat3+e3[i]+'/')))


for i in range(len(x)):
  if x[i] ==".ipynb_checkpoints":
    x.remove(x[i])
    break
for j in range(len(y)):
  for i in range(len(y[j])):
    if y[j][i] ==".ipynb_checkpoints":
      y[j].remove(y[j][i])
      break
cls=[]

def dp(dist_mat):
    """
    Find minimum-cost path through matrix dist_mat using dynamic programming.

    The cost of a path is defined as the sum of the matrix entries on that
    path. See the following for details of the algorithm:

    - http://en.wikipedia.org/wiki/Dynamic_time_warping
    - https://www.ee.columbia.edu/~dpwe/resources/matlab/dtw/dp.m

    The notation in the first reference was followed, while Dan Ellis's code
    (second reference) was used to check for correctness. Returns a list of
    path indices and the cost matrix.
    """

    N, M = dist_mat.shape

    # Initialize the cost matrix
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1, N + 1):
        cost_mat[i, 0] = np.inf
    for i in range(1, M + 1):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_mat[i, j],  # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = N - 1
    j = M - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))

    # Strip infinity edges from cost_mat before returning
    cost_mat = cost_mat[1:, 1:]
    return (path[::-1], cost_mat)




# Distance matrix
def subf( a , b,pat2):
    f=open(pat+a,'r')
    x3= json.loads( f.read())
    orign=x3['part_candidates'][0]
    f.close()

    f=open(pat2+b,'r')
    x2= json.loads( f.read())
    new=x2['part_candidates'][0]

    f.close()
    const=0
    avg=0
 

    for i in range(25):
        if (orign[str(i)] != [] and len(orign[str(i)]) >= 2) and (new[str(i)] != [] and len(new[str(i)]) >= 2):
            const += sqrt((orign[str(i)][0] - new[str(i)][0]) ** 2 + (orign[str(i)][1] - new[str(i)][1]) ** 2)
            avg= avg+1



    const=const/avg
    return const






pa=pat1
e=e1
for c in range(len(y)):
  if(c==5):
    pa=pat2
    e=e2
  if(c==10):
    pa=pat3
    e=e3
  dist_mat = np.zeros((len(x), len(y[c])))
  for i in range(len(x)):
        for j in range(len(y[c])):
            print(x[i],y[c][j],pa+e[c%5]+'/')
            dist_mat[i, j]=subf(x[i],y[c][j],pa+e[c%5]+'/')

  path, cost_mat = dp(dist_mat)

  cls.append(cost_mat[len(cost_mat)-1][len(cost_mat[0])-1])









if(cls.index(min(cls)) < 5):
  print("==============================================================================")
  print("The first exercise")
  print("==============================================================================")

elif(cls.index(min(cls)) < 10):
  print("==============================================================================")
  print("The secound exercise")
  print("==============================================================================")
else:
  print("==============================================================================")
  print("The Third exercise")
  print("==============================================================================")

#cls.index(min(cls))



#a=[3,1,1,3,4,5]

#print(a.index(min(a)))
#a.remove(1)
#print(a)
#a.pop(5)
######################################

Streaming output truncated to the last 5000 lines.
TestingVid8_000000000275_keypoints.json Jogging5_000000000152_keypoints.json /content/drive/MyDrive/Jogging/Jogging5/
TestingVid8_000000000275_keypoints.json Jogging5_000000000153_keypoints.json /content/drive/MyDrive/Jogging/Jogging5/
TestingVid8_000000000275_keypoints.json Jogging5_000000000154_keypoints.json /content/drive/MyDrive/Jogging/Jogging5/
TestingVid8_000000000275_keypoints.json Jogging5_000000000155_keypoints.json /content/drive/MyDrive/Jogging/Jogging5/
TestingVid8_000000000275_keypoints.json Jogging5_000000000156_keypoints.json /content/drive/MyDrive/Jogging/Jogging5/
TestingVid8_000000000275_keypoints.json Jogging5_000000000157_keypoints.json /content/drive/MyDrive/Jogging/Jogging5/
TestingVid8_000000000275_keypoints.json Jogging5_000000000158_keypoints.json /content/drive/MyDrive/Jogging/Jogging5/
TestingVid8_000000000275_keypoints.json Jogging5_000000000159_keypoints.json /content/drive/MyDrive/Jogging/Jogging5/
Testi

In [ ]:

# BaryCenter calculations

from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import numpy as np
import os
import json
from math import sqrt
from google.colab import drive

drive.mount('/content/drive')
pat="/content/out/"#todoooooooooooooooooooooo
pat1="/content/e2/"#todooooooooooooooooo

x = sorted(os.listdir(pat))#['ba)tata.json', 'sese.json']
e1=sorted(os.listdir(pat1))


y = []

for i in range(len(e1)):
  y.append(sorted(os.listdir(pat1+e1[i]+'/')))


for i in range(len(x)):
  if x[i] ==".ipynb_checkpoints":
    x.remove(x[i])
    break
for j in range(len(y)):
  for i in range(len(y[j])):
    if y[j][i] ==".ipynb_checkpoints":
      y[j].remove(y[j][i])
      break
cls=[]
paths=[]

def dp(dist_mat):
    """
    Find minimum-cost path through matrix dist_mat using dynamic programming.

    The cost of a path is defined as the sum of the matrix entries on that
    path. See the following for details of the algorithm:

    - http://en.wikipedia.org/wiki/Dynamic_time_warping
    - https://www.ee.columbia.edu/~dpwe/resources/matlab/dtw/dp.m

    The notation in the first reference was followed, while Dan Ellis's code
    (second reference) was used to check for correctness. Returns a list of
    path indices and the cost matrix.
    """

    N, M = dist_mat.shape

    # Initialize the cost matrix
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1, N + 1):
        cost_mat[i, 0] = np.inf
    for i in range(1, M + 1):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_mat[i, j],  # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = N - 1
    j = M - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))

    # Strip infinity edges from cost_mat before returning
    cost_mat = cost_mat[1:, 1:]
    return (path[::-1], cost_mat)


def par( a , b,pat2):
    f=open(pat2+b,'r')
    x2= json.loads( f.read())
    new=x2['part_candidates'][0]
    f.close()

   

    f=open(pat+a,'r+')
    x3= json.loads( f.read())
    orign=x3['part_candidates'][0]
    print(orign[str(0)][0])
    print("================================================")
    for i in range(25):
        if (orign[str(i)] != [] and len(orign[str(i)]) >= 2) and (new[str(i)] != [] and len(new[str(i)]) >= 2):
            orign[str(i)][0] =(orign[str(i)][0] + new[str(i)][0])
            orign[str(i)][1] =(orign[str(i)][1] + new[str(i)][1])
    print(orign[str(0)][0],new[str(0)][0])
    jprint=json.dumps(orign)
    f.seek(0)
    f.write('{"part_candidates":[')
    f.write(jprint)
    f.write("]}")
    f.truncate()
    f.close()
    return


# Distance matrix
def subf( a , b,pat2):
    f=open(pat+a,'r')
    x3= json.loads( f.read())
    orign=x3['part_candidates'][0]
    f.close()

    f=open(pat2+b,'r')
    x2= json.loads( f.read())
    new=x2['part_candidates'][0]

    f.close()
    const=0
    avg=0
 

    for i in range(25):
        if (orign[str(i)] != [] and len(orign[str(i)]) >= 2) and (new[str(i)] != [] and len(new[str(i)]) >= 2):
            const += sqrt((orign[str(i)][0] - new[str(i)][0]) ** 2 + (orign[str(i)][1] - new[str(i)][1]) ** 2)
            avg= avg+1



    const=const/avg
    return const






pa=pat1
e=e1
for c in range(len(y)):

  dist_mat = np.zeros((len(x), len(y[c])))
  for i in range(len(x)):
        for j in range(len(y[c])):
            print(x[i],y[c][j],pa+e[c]+'/')
            dist_mat[i, j]=subf(x[i],y[c][j],pa+e[c]+'/')

  path, cost_mat = dp(dist_mat)

 
  paths.append(path)



print(x[paths[0][0][0]],y[0][paths[0][0][1]])
e=e1
print(paths)


count=[1]*len(x)
for i in range(len(paths)):
  for j in range(len(paths[i])):
 
    par(x[paths[i][j][0]],y[i][paths[i][j][1]],pat1+e[i]+'/')
    count[paths[i][j][0]]=count[paths[i][j][0]]+1
    

print(count)

for j in range(len(x)):
  f=open(pat+x[j],'r+')
  x3= json.loads( f.read())
  orign=x3['part_candidates'][0]
  print(count[j])  
  for i in range(25):
      if (orign[str(i)] != [] and len(orign[str(i)]) >= 2):
          orign[str(i)][0] =(orign[str(i)][0] )/count[j]
          orign[str(i)][1] =(orign[str(i)][1])/count[j]
  jprint=json.dumps(orign)
  f.seek(0)
  f.write('{"part_candidates":[')
  f.write(jprint)
  f.write("]}")
  f.truncate()
  f.close()





#cls.index(min(cls))



#a=[3,1,1,3,4,5]

#print(a.index(min(a)))
#a.remove(1)
#print(a)
#a.pop(5)
######################################

Streaming output truncated to the last 5000 lines.
0.11920329322891748
0.1635092559316161 0.044305962702698626
0.07841867144844751
0.10974979647307465 0.031331125024627135
0.0819688468133265
0.11408035038363254 0.032111503570306045
0.06837396821444164
0.10073511197637544 0.0323611437619338
0.06860299199445909
0.1032187907882156 0.03461579879375651
0.07449311357868971
0.12768705834599275 0.05319394476730306
0.08982425203721858
0.14301819680452166 0.05319394476730306
0.07431755233900411
0.12751149710630716 0.05319394476730306
0.1178301801753535
0.17102412494265656 0.05319394476730306
0.11493153962277944
0.1681254843900825 0.05319394476730306
0.11193115338258601
0.16512509814988907 0.05319394476730306
0.12622740046017203
0.1794213452274751 0.05319394476730306
0.17169056383564857
0.24228934177519015 0.07059877793954158
0.136598111631339
0.2071968895708806 0.07059877793954158
0.1389172265418441
0.2095160044813857 0.07059877793954158
0.10957104897326689
0.18016982691280847 0.0705987779395415